In [1]:
from numpy import mean
from numpy import std
from numpy import absolute
import pandas as pd
# use automatically configured elastic net algorithm
from numpy import arange
from sklearn.model_selection import cross_val_score
#from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import ElasticNet
#from sklearn.linear_model import ElasticNet
from sklearn.model_selection import RepeatedKFold, cross_val_predict
from sklearn.metrics import mean_absolute_error, r2_score
from scipy.stats import pearsonr
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import LeaveOneOut
import matplotlib.pyplot as plt
import statsmodels.api as sm
lowess = sm.nonparametric.lowess
from sklearn.model_selection import GridSearchCV
import seaborn as sns
from scipy import stats
import os
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA

In [2]:
os.chdir("/home/lajoyce/Documents/killifish_vs_mouse/analysis/mouse")

In [3]:
df = pd.read_csv("/home/lajoyce/Documents/killifish_vs_mouse/analysis/mouse/MACA_modified_rawcounts.csv")

df.head(10)

,Unnamed: 0,Brain_47-m-18,GAT_39-f-1,Lung_6-f-12,Limb_Muscle_11-m-3,WBC_34-f-9,Small_Intestine_46-m-12,Limb_Muscle_38-m-9,Liver_56-m-6,Lung_17-f-12,...,Spleen_25-m-9,Pancreas_14-m-27,BAT_27-f-6,Kidney_7-m-15,Brain_8-f-12,Brain_22-m-15,WBC_44-f-12,WBC_29-m-9,Spleen_33-f-1,WBC_23-m-3
0,ENSMUSG00000109644,8,29,5,6,3,477,4,2494,7,...,19,7,5,1004,8,16,5,1,9,0
1,ENSMUSG00000007777,118,269,117,209,2,215,96,214,91,...,49,29,1118,136,178,161,3,10,112,2
2,ENSMUSG00000043644,17,40,21,148,9,12,88,12,42,...,24,0,36,14,15,16,15,15,33,7
3,ENSMUSG00000042208,546,570,291,259,41,93,195,327,230,...,152,49,1853,79,657,781,11,17,265,5
4,ENSMUSG00000020831,39,113,40,19,8,14,9,21,33,...,75,11,56,27,56,76,1,4,83,0
5,ENSMUSG00000107002,284,488,204,635,31,537,475,346,183,...,235,88,3577,297,489,577,23,26,347,18
6,ENSMUSG00000058706,10,87,40,24,21,40,25,144,59,...,46,14,85,22,32,27,6,6,141,0
7,ENSMUSG00000099146,18,0,9,0,3,2,3,630,8,...,3,1,0,68,8,7,0,0,3,0
8,ENSMUSG00000028608,419,532,471,222,78,193,190,193,510,...,432,103,916,162,669,647,27,40,651,16
9,ENSMUSG00000097882,19,44,13,30,0,0,14,1,29,...,25,9,6,11,32,16,0,1,20,1


In [4]:
df.rename(columns={"Unnamed: 0": "Gene"}, inplace=True)

In [5]:
df.set_index("Gene", inplace=True)

In [6]:
df = df.T

In [7]:
df.shape

(892, 24616)

In [8]:
df_metadata = pd.read_csv("MACA_modified_metadata.csv")

df_metadata.head(1)

,Unnamed: 0,Sample.name,title,source.name,organism,characteristics..age,characteristics..developmental.stage,characteristics..sex,molecule,description,processed.data.file,raw.file,BioSample,Instrument.Model,Sample.name.new
0,A1_384Bulk_Plate1_S1,A1_384Bulk_Plate1_S1,Tabula Muris Senis (bulk RNA seq),BAT_24,Mus musculus C57/BL6,6,months postnatal,m,total RNA,NaN,190214_A00111_0269_AHH3J3DSXX__190214_A00111_0...,SRR9127205,SAMN11854566,Illumina NovaSeq 6000,BAT_24-m-6


In [9]:
df_metadata.set_index("Sample.name.new", inplace=True)

In [10]:
df_new_metadata = df_metadata.loc[df.index.tolist(), ]

df_new_metadata.head(5)

,Unnamed: 0,Sample.name,title,source.name,organism,characteristics..age,characteristics..developmental.stage,characteristics..sex,molecule,description,processed.data.file,raw.file,BioSample,Instrument.Model
Sample.name.new,,,,,,,,,,,,,,
Brain_47-m-18,A10_384Bulk_Plate1_S10,A10_384Bulk_Plate1_S10,Tabula Muris Senis (bulk RNA seq),Brain_47,Mus musculus C57/BL6,18,months postnatal,m,total RNA,NaN,190214_A00111_0269_AHH3J3DSXX__190214_A00111_0...,SRR9127203,SAMN11854568,Illumina NovaSeq 6000
GAT_39-f-1,A10_384Bulk_Plate2_S10,A10_384Bulk_Plate2_S10,Tabula Muris Senis (bulk RNA seq),GAT_39,Mus musculus C57/BL6,1,months postnatal,f,total RNA,NaN,190214_A00111_0269_AHH3J3DSXX__190214_A00111_0...,SRR9127204,SAMN11854569,Illumina NovaSeq 6000
Lung_6-f-12,A10_384Bulk_Plate3_S10,A10_384Bulk_Plate3_S10,Tabula Muris Senis (bulk RNA seq),Lung_6,Mus musculus C57/BL6,12,months postnatal,f,total RNA,NaN,190214_A00111_0269_AHH3J3DSXX__190214_A00111_0...,SRR9127371,SAMN11854570,Illumina NovaSeq 6000
Limb_Muscle_11-m-3,A11_384Bulk_Plate1_S11,A11_384Bulk_Plate1_S11,Tabula Muris Senis (bulk RNA seq),Limb_Muscle_11,Mus musculus C57/BL6,3,months postnatal,m,total RNA,NaN,190214_A00111_0269_AHH3J3DSXX__190214_A00111_0...,SRR9127373,SAMN11854571,Illumina NovaSeq 6000
WBC_34-f-9,A11_384Bulk_Plate2_S11,A11_384Bulk_Plate2_S11,Tabula Muris Senis (bulk RNA seq),WBC_34,Mus musculus C57/BL6,9,months postnatal,f,total RNA,NaN,190214_A00111_0269_AHH3J3DSXX__190214_A00111_0...,SRR9127107,SAMN11854572,Illumina NovaSeq 6000


# BAT

In [11]:
bat_df_metadata = df_new_metadata[df_new_metadata["source.name"].str.startswith("BAT")]

bat_df_metadata.head(3)

,Unnamed: 0,Sample.name,title,source.name,organism,characteristics..age,characteristics..developmental.stage,characteristics..sex,molecule,description,processed.data.file,raw.file,BioSample,Instrument.Model
Sample.name.new,,,,,,,,,,,,,,
BAT_24-m-6,A1_384Bulk_Plate1_S1,A1_384Bulk_Plate1_S1,Tabula Muris Senis (bulk RNA seq),BAT_24,Mus musculus C57/BL6,6,months postnatal,m,total RNA,NaN,190214_A00111_0269_AHH3J3DSXX__190214_A00111_0...,SRR9127205,SAMN11854566,Illumina NovaSeq 6000
BAT_34-m-27,A2_384Bulk_Plate3_S2,A2_384Bulk_Plate3_S2,Tabula Muris Senis (bulk RNA seq),BAT_34,Mus musculus C57/BL6,27,months postnatal,m,total RNA,NaN,190214_A00111_0269_AHH3J3DSXX__190214_A00111_0...,SRR9127269,SAMN11854595,Illumina NovaSeq 6000
BAT_43-m-21,A4_384Bulk_Plate3_S4,A4_384Bulk_Plate3_S4,Tabula Muris Senis (bulk RNA seq),BAT_43,Mus musculus C57/BL6,21,months postnatal,m,total RNA,NaN,190214_A00111_0269_AHH3J3DSXX__190214_A00111_0...,SRR9127252,SAMN11854609,Illumina NovaSeq 6000


In [12]:
bat_df_counts = df.loc[bat_df_metadata.index.tolist(),]

bat_df_counts.head(3)

Gene,ENSMUSG00000109644,ENSMUSG00000007777,ENSMUSG00000043644,ENSMUSG00000042208,ENSMUSG00000020831,ENSMUSG00000107002,ENSMUSG00000058706,ENSMUSG00000099146,ENSMUSG00000028608,ENSMUSG00000097882,...,ENSMUSG00000096015,ENSMUSG00000096037,ENSMUSG00000095183,ENSMUSG00000094760,ENSMUSG00000095749,ENSMUSG00000094657,ENSMUSG00000095645,ENSMUSG00000096765,ENSMUSG00000096170,ENSMUSG00000095453
BAT_24-m-6,4,118,2,208,13,550,16,0,152,1,...,0,0,0,0,0,0,0,0,0,0
BAT_34-m-27,6,229,14,445,21,1071,28,5,318,7,...,0,0,0,0,0,0,0,0,0,0
BAT_43-m-21,2,197,19,315,12,952,38,0,274,4,...,0,0,0,0,0,0,0,0,0,0


In [13]:
bat_df_age = bat_df_counts.copy()

In [14]:
bat_df_age["age"] = bat_df_metadata["characteristics..age"].tolist()

In [15]:
bat_df_age["age"] = pd.to_numeric(bat_df_metadata["characteristics..age"].tolist(), errors="coerce")

In [16]:
bat_log_transformed_data = np.log1p(bat_df_counts)

In [17]:
data = bat_df_age.values

In [18]:
X = bat_log_transformed_data.values
y = data[:,-1]

In [19]:
# Define the parameter grid
param_grid = {
    'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1.0, 10.0, 100.0],
    'l1_ratio': np.arange(0, 1.1, 0.1)
}

# Set a seed for reproducibility
seed = 42

# Define the model with a random seed
model = ElasticNet(max_iter=100000, random_state=seed)

# Define Leave-One-Out cross-validation
loo = LeaveOneOut()

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=loo, scoring='neg_mean_absolute_error', n_jobs=-1, return_train_score=True)

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Fit the GridSearchCV
grid_search.fit(X_scaled, y)

# Get the best parameters
best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

# Retrieve the results of the grid search
results = pd.DataFrame(grid_search.cv_results_)

# Convert the negative MAE to positive for clarity
results['mean_test_MAE'] = -results['mean_test_score']

# Save the results to a CSV file or print them
results_to_save = results[['param_alpha', 'param_l1_ratio', 'mean_test_MAE']]
results_to_save.to_csv('/home/lajoyce/Documents/BayesAge2.0/elastic_net/BAT_elastic_net_tuning_results.csv', index=False)
print("MAE under each combination of alpha and l1_ratio:")
print(results_to_save)

# Train the model with the best parameters using Leave-One-Out cross-validation
y_true = []
y_pred = []

for train_index, test_index in loo.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Use the same scaling during LOOCV
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Train ElasticNet with the best parameters and the random seed
    best_model = ElasticNet(alpha=best_params['alpha'], l1_ratio=best_params['l1_ratio'], max_iter=10000, random_state=seed)
    best_model.fit(X_train_scaled, y_train)
    
    # Predict the test sample
    y_test_pred = best_model.predict(X_test_scaled)
    
    y_true.append(y_test[0])
    y_pred.append(y_test_pred[0])

# Convert lists to numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Calculate the Pearson correlation coefficient
pearson_corr, _ = pearsonr(y_true, y_pred)

# Calculate the R-squared value
r_squared = r2_score(y_true, y_pred)

# Calculate the mean absolute error
mae = mean_absolute_error(y_true, y_pred)

print(f"Mean MAE: {mae:.3f}")
print(f"Pearson correlation: {pearson_corr:.3f}")
print(f"R-squared: {r_squared:.3f}")

/home/lajoyce/Documents/USQIS_2023/qiskit/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.445e-01, tolerance: 3.143e-01 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/home/lajoyce/Documents/USQIS_2023/qiskit/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.586e-01, tolerance: 3.262e-01 Linear regression models with null weight for the l1 regularization term are more efficiently fitt

Best parameters found:  {'alpha': 0.001, 'l1_ratio': 1.0}
MAE under each combination of alpha and l1_ratio:
   param_alpha param_l1_ratio  mean_test_MAE
0      0.00001            0.0       8.194781
1      0.00001            0.1       8.200403
2      0.00001            0.2       8.205823
3      0.00001            0.3       8.205821
4      0.00001            0.4       8.205823
..         ...            ...            ...
83       100.0            0.6       6.977504
84       100.0            0.7       6.977504
85       100.0            0.8       6.977504
86       100.0            0.9       6.977504
87       100.0            1.0       6.977504

[88 rows x 3 columns]
Mean MAE: 2.489
Pearson correlation: 0.927
R-squared: 0.841


In [20]:
loo = LeaveOneOut()

# Initialize lists to store actual and predicted values
y_true = []
y_pred = []

# Loop over each train-test split
for train_index, test_index in loo.split(X):
    # Split the data
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Scale the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Define and train the model
    model = ElasticNet(alpha=0.001, l1_ratio=1.0, max_iter=100000, random_state=seed)
    model.fit(X_train_scaled, y_train)
    #model.fit(X_train, y_train)
    
    # Make prediction on the test set
    y_test_pred = model.predict(X_test_scaled)
    #y_test_pred = model.predict(X_test)
    
    # Store the actual and predicted values
    y_true.append(y_test[0])
    y_pred.append(y_test_pred[0])

# Convert lists to numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Calculate the Pearson correlation coefficient
pearson_corr, _ = pearsonr(y_true, y_pred)

# Calculate the R-squared value
r_squared = r2_score(y_true, y_pred)

# Calculate the mean absolute error
mae = mean_absolute_error(y_true, y_pred)

print(f"Mean MAE: {mae:.3f}")
print(f"Pearson correlation: {pearson_corr:.3f}")
print(f"R-squared: {r_squared:.3f}")

Mean MAE: 2.489
Pearson correlation: 0.927
R-squared: 0.841


# Bone

In [21]:
bone_df_metadata = df_new_metadata[df_new_metadata["source.name"].str.startswith("Bone")]

bone_df_metadata.head(3)

,Unnamed: 0,Sample.name,title,source.name,organism,characteristics..age,characteristics..developmental.stage,characteristics..sex,molecule,description,processed.data.file,raw.file,BioSample,Instrument.Model
Sample.name.new,,,,,,,,,,,,,,
Bone_15-m-15,A14_384Bulk_Plate1_S14,A14_384Bulk_Plate1_S14,Tabula Muris Senis (bulk RNA seq),Bone_15,Mus musculus C57/BL6,15,months postnatal,m,total RNA,NaN,190214_A00111_0269_AHH3J3DSXX__190214_A00111_0...,SRR9127024,SAMN11854580,Illumina NovaSeq 6000
Bone_45-m-15,A8_384Bulk_Plate1_S8,A8_384Bulk_Plate1_S8,Tabula Muris Senis (bulk RNA seq),Bone_45,Mus musculus C57/BL6,15,months postnatal,m,total RNA,NaN,190214_A00111_0269_AHH3J3DSXX__190214_A00111_0...,SRR9127051,SAMN11854619,Illumina NovaSeq 6000
Bone_36-f-6,B13_384Bulk_Plate1_S37,B13_384Bulk_Plate1_S37,Tabula Muris Senis (bulk RNA seq),Bone_36,Mus musculus C57/BL6,6,months postnatal,f,total RNA,NaN,190214_A00111_0269_AHH3J3DSXX__190214_A00111_0...,SRR9126850,SAMN11854637,Illumina NovaSeq 6000


In [22]:
bone_df_counts = df.loc[bone_df_metadata.index.tolist(),]

bone_df_counts.head(3)

Gene,ENSMUSG00000109644,ENSMUSG00000007777,ENSMUSG00000043644,ENSMUSG00000042208,ENSMUSG00000020831,ENSMUSG00000107002,ENSMUSG00000058706,ENSMUSG00000099146,ENSMUSG00000028608,ENSMUSG00000097882,...,ENSMUSG00000096015,ENSMUSG00000096037,ENSMUSG00000095183,ENSMUSG00000094760,ENSMUSG00000095749,ENSMUSG00000094657,ENSMUSG00000095645,ENSMUSG00000096765,ENSMUSG00000096170,ENSMUSG00000095453
Bone_15-m-15,26,39,5,96,31,49,26,25,123,8,...,0,0,0,0,0,0,0,0,0,0
Bone_45-m-15,35,27,22,163,19,46,8,26,172,27,...,0,0,0,0,0,0,0,0,0,0
Bone_36-f-6,16,63,9,141,30,141,24,9,158,20,...,0,0,0,0,0,0,0,0,0,0


In [23]:
bone_df_age = bone_df_counts.copy()

In [24]:
bone_df_age["age"] = bone_df_metadata["characteristics..age"].tolist()

In [25]:
bone_df_age.head(5)

Gene,ENSMUSG00000109644,ENSMUSG00000007777,ENSMUSG00000043644,ENSMUSG00000042208,ENSMUSG00000020831,ENSMUSG00000107002,ENSMUSG00000058706,ENSMUSG00000099146,ENSMUSG00000028608,ENSMUSG00000097882,...,ENSMUSG00000096037,ENSMUSG00000095183,ENSMUSG00000094760,ENSMUSG00000095749,ENSMUSG00000094657,ENSMUSG00000095645,ENSMUSG00000096765,ENSMUSG00000096170,ENSMUSG00000095453,age
Bone_15-m-15,26,39,5,96,31,49,26,25,123,8,...,0,0,0,0,0,0,0,0,0,15
Bone_45-m-15,35,27,22,163,19,46,8,26,172,27,...,0,0,0,0,0,0,0,0,0,15
Bone_36-f-6,16,63,9,141,30,141,24,9,158,20,...,0,0,0,0,0,0,0,0,0,6
Bone_4-f-15,4,36,7,81,17,82,13,12,100,9,...,0,0,0,0,0,0,0,0,0,15
Bone_10-m-12,8,85,19,203,40,141,142,7,259,12,...,0,0,0,0,0,0,0,0,0,12


In [26]:
bone_df_age["age"] = pd.to_numeric(bone_df_metadata["characteristics..age"].tolist(), errors="coerce")

In [27]:
bone_df_age.head(3)

Gene,ENSMUSG00000109644,ENSMUSG00000007777,ENSMUSG00000043644,ENSMUSG00000042208,ENSMUSG00000020831,ENSMUSG00000107002,ENSMUSG00000058706,ENSMUSG00000099146,ENSMUSG00000028608,ENSMUSG00000097882,...,ENSMUSG00000096037,ENSMUSG00000095183,ENSMUSG00000094760,ENSMUSG00000095749,ENSMUSG00000094657,ENSMUSG00000095645,ENSMUSG00000096765,ENSMUSG00000096170,ENSMUSG00000095453,age
Bone_15-m-15,26,39,5,96,31,49,26,25,123,8,...,0,0,0,0,0,0,0,0,0,15
Bone_45-m-15,35,27,22,163,19,46,8,26,172,27,...,0,0,0,0,0,0,0,0,0,15
Bone_36-f-6,16,63,9,141,30,141,24,9,158,20,...,0,0,0,0,0,0,0,0,0,6


In [28]:
bone_log_transformed_data = np.log1p(bone_df_counts)

In [29]:
bone_data = bone_df_age.values

In [30]:
X = bone_log_transformed_data.values
y = bone_data[:,-1]

In [31]:
# Define the parameter grid
param_grid = {
    'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1.0, 10.0, 100.0],
    'l1_ratio': np.arange(0, 1.1, 0.1)
}

# Set a seed for reproducibility
seed = 42

# Define the model with a random seed
model = ElasticNet(max_iter=100000, random_state=seed)

# Define Leave-One-Out cross-validation
loo = LeaveOneOut()

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=loo, scoring='neg_mean_absolute_error', n_jobs=-1, return_train_score=True)

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Fit the GridSearchCV
grid_search.fit(X_scaled, y)

# Get the best parameters
best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

# Retrieve the results of the grid search
results = pd.DataFrame(grid_search.cv_results_)

# Convert the negative MAE to positive for clarity
results['mean_test_MAE'] = -results['mean_test_score']

# Save the results to a CSV file or print them
results_to_save = results[['param_alpha', 'param_l1_ratio', 'mean_test_MAE']]
results_to_save.to_csv('/home/lajoyce/Documents/BayesAge2.0/elastic_net/Bone_elastic_net_tuning_results.csv', index=False)
print("MAE under each combination of alpha and l1_ratio:")
print(results_to_save)

# Train the model with the best parameters using Leave-One-Out cross-validation
y_true = []
y_pred = []

for train_index, test_index in loo.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Use the same scaling during LOOCV
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Train ElasticNet with the best parameters and the random seed
    best_model = ElasticNet(alpha=best_params['alpha'], l1_ratio=best_params['l1_ratio'], max_iter=10000, random_state=seed)
    best_model.fit(X_train_scaled, y_train)
    
    # Predict the test sample
    y_test_pred = best_model.predict(X_test_scaled)
    
    y_true.append(y_test[0])
    y_pred.append(y_test_pred[0])

# Convert lists to numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Calculate the Pearson correlation coefficient
pearson_corr, _ = pearsonr(y_true, y_pred)

# Calculate the R-squared value
r_squared = r2_score(y_true, y_pred)

# Calculate the mean absolute error
mae = mean_absolute_error(y_true, y_pred)

print(f"Mean MAE: {mae:.3f}")
print(f"Pearson correlation: {pearson_corr:.3f}")
print(f"R-squared: {r_squared:.3f}")

/home/lajoyce/Documents/USQIS_2023/qiskit/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.040e-01, tolerance: 3.321e-01 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/home/lajoyce/Documents/USQIS_2023/qiskit/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.106e-01, tolerance: 3.437e-01 Linear regression models with null weight for the l1 regularization term are more efficiently fitt

Best parameters found:  {'alpha': 0.01, 'l1_ratio': 0.30000000000000004}
MAE under each combination of alpha and l1_ratio:
   param_alpha param_l1_ratio  mean_test_MAE
0      0.00001            0.0       5.680203
1      0.00001            0.1       5.680201
2      0.00001            0.2       5.680200
3      0.00001            0.3       5.680199
4      0.00001            0.4       5.680198
..         ...            ...            ...
83       100.0            0.6       7.024458
84       100.0            0.7       7.024458
85       100.0            0.8       7.024458
86       100.0            0.9       7.024458
87       100.0            1.0       7.024458

[88 rows x 3 columns]


/home/lajoyce/Documents/USQIS_2023/qiskit/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.888e-01, tolerance: 3.389e-01
  model = cd_fast.enet_coordinate_descent(
/home/lajoyce/Documents/USQIS_2023/qiskit/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.354e-01, tolerance: 3.290e-01
  model = cd_fast.enet_coordinate_descent(
/home/lajoyce/Documents/USQIS_2023/qiskit/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of 

Mean MAE: 3.899
Pearson correlation: 0.836
R-squared: 0.637


In [34]:
loo = LeaveOneOut()

# Initialize lists to store actual and predicted values
y_true = []
y_pred = []

# Loop over each train-test split
for train_index, test_index in loo.split(X):
    # Split the data
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Scale the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Define and train the model
    model = ElasticNet(alpha=0.01, l1_ratio=0.30000000000000004, max_iter=100000, random_state=seed)
    model.fit(X_train_scaled, y_train)
    #model.fit(X_train, y_train)
    
    # Make prediction on the test set
    y_test_pred = model.predict(X_test_scaled)
    #y_test_pred = model.predict(X_test)
    
    # Store the actual and predicted values
    y_true.append(y_test[0])
    y_pred.append(y_test_pred[0])

# Convert lists to numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Calculate the Pearson correlation coefficient
pearson_corr, _ = pearsonr(y_true, y_pred)

# Calculate the R-squared value
r_squared = r2_score(y_true, y_pred)

# Calculate the mean absolute error
mae = mean_absolute_error(y_true, y_pred)

print(f"Mean MAE: {mae:.3f}")
print(f"Pearson correlation: {pearson_corr:.3f}")
print(f"R-squared: {r_squared:.3f}")

Mean MAE: 3.906
Pearson correlation: 0.833
R-squared: 0.633


# Brain

In [35]:
brain_df_metadata = df_new_metadata[df_new_metadata["source.name"].str.startswith("Brain")]

brain_df_metadata.head(3)

,Unnamed: 0,Sample.name,title,source.name,organism,characteristics..age,characteristics..developmental.stage,characteristics..sex,molecule,description,processed.data.file,raw.file,BioSample,Instrument.Model
Sample.name.new,,,,,,,,,,,,,,
Brain_47-m-18,A10_384Bulk_Plate1_S10,A10_384Bulk_Plate1_S10,Tabula Muris Senis (bulk RNA seq),Brain_47,Mus musculus C57/BL6,18,months postnatal,m,total RNA,NaN,190214_A00111_0269_AHH3J3DSXX__190214_A00111_0...,SRR9127203,SAMN11854568,Illumina NovaSeq 6000
Brain_51-f-18,A13_384Bulk_Plate3_S13,A13_384Bulk_Plate3_S13,Tabula Muris Senis (bulk RNA seq),Brain_51,Mus musculus C57/BL6,18,months postnatal,f,total RNA,NaN,190214_A00111_0269_AHH3J3DSXX__190214_A00111_0...,SRR9127027,SAMN11854579,Illumina NovaSeq 6000
Brain_20-f-21,A18_384Bulk_Plate1_S18,A18_384Bulk_Plate1_S18,Tabula Muris Senis (bulk RNA seq),Brain_20,Mus musculus C57/BL6,21,months postnatal,f,total RNA,NaN,190214_A00111_0269_AHH3J3DSXX__190214_A00111_0...,SRR9127073,SAMN11854589,Illumina NovaSeq 6000


In [36]:
brain_df_counts = df.loc[brain_df_metadata.index.tolist(),]

brain_df_counts.head(3)

Gene,ENSMUSG00000109644,ENSMUSG00000007777,ENSMUSG00000043644,ENSMUSG00000042208,ENSMUSG00000020831,ENSMUSG00000107002,ENSMUSG00000058706,ENSMUSG00000099146,ENSMUSG00000028608,ENSMUSG00000097882,...,ENSMUSG00000096015,ENSMUSG00000096037,ENSMUSG00000095183,ENSMUSG00000094760,ENSMUSG00000095749,ENSMUSG00000094657,ENSMUSG00000095645,ENSMUSG00000096765,ENSMUSG00000096170,ENSMUSG00000095453
Brain_47-m-18,8,118,17,546,39,284,10,18,419,19,...,0,0,0,0,0,0,0,0,0,0
Brain_51-f-18,8,67,11,251,26,199,9,1,232,9,...,0,0,0,0,0,0,0,0,0,0
Brain_20-f-21,9,138,19,640,56,381,12,18,482,23,...,0,0,0,0,0,0,0,0,0,0


In [37]:
brain_df_age = brain_df_counts.copy()

In [38]:
brain_df_age["age"] = brain_df_metadata["characteristics..age"].tolist()

In [39]:
brain_df_age.head(5)

Gene,ENSMUSG00000109644,ENSMUSG00000007777,ENSMUSG00000043644,ENSMUSG00000042208,ENSMUSG00000020831,ENSMUSG00000107002,ENSMUSG00000058706,ENSMUSG00000099146,ENSMUSG00000028608,ENSMUSG00000097882,...,ENSMUSG00000096037,ENSMUSG00000095183,ENSMUSG00000094760,ENSMUSG00000095749,ENSMUSG00000094657,ENSMUSG00000095645,ENSMUSG00000096765,ENSMUSG00000096170,ENSMUSG00000095453,age
Brain_47-m-18,8,118,17,546,39,284,10,18,419,19,...,0,0,0,0,0,0,0,0,0,18
Brain_51-f-18,8,67,11,251,26,199,9,1,232,9,...,0,0,0,0,0,0,0,0,0,18
Brain_20-f-21,9,138,19,640,56,381,12,18,482,23,...,0,0,0,0,0,0,0,0,0,21
Brain_39-m-12,22,133,23,593,71,395,4,17,439,12,...,0,0,0,0,0,0,0,0,0,12
Brain_4-m-3,11,148,23,907,44,451,28,6,577,7,...,0,0,0,0,0,0,0,0,0,3


In [40]:
brain_df_age["age"] = pd.to_numeric(brain_df_metadata["characteristics..age"].tolist(), errors="coerce")

In [41]:
brain_df_age.head(3)

Gene,ENSMUSG00000109644,ENSMUSG00000007777,ENSMUSG00000043644,ENSMUSG00000042208,ENSMUSG00000020831,ENSMUSG00000107002,ENSMUSG00000058706,ENSMUSG00000099146,ENSMUSG00000028608,ENSMUSG00000097882,...,ENSMUSG00000096037,ENSMUSG00000095183,ENSMUSG00000094760,ENSMUSG00000095749,ENSMUSG00000094657,ENSMUSG00000095645,ENSMUSG00000096765,ENSMUSG00000096170,ENSMUSG00000095453,age
Brain_47-m-18,8,118,17,546,39,284,10,18,419,19,...,0,0,0,0,0,0,0,0,0,18
Brain_51-f-18,8,67,11,251,26,199,9,1,232,9,...,0,0,0,0,0,0,0,0,0,18
Brain_20-f-21,9,138,19,640,56,381,12,18,482,23,...,0,0,0,0,0,0,0,0,0,21


In [42]:
brain_log_transformed_data = np.log1p(brain_df_counts)

In [43]:
brain_log_transformed_data.head(5)

Gene,ENSMUSG00000109644,ENSMUSG00000007777,ENSMUSG00000043644,ENSMUSG00000042208,ENSMUSG00000020831,ENSMUSG00000107002,ENSMUSG00000058706,ENSMUSG00000099146,ENSMUSG00000028608,ENSMUSG00000097882,...,ENSMUSG00000096015,ENSMUSG00000096037,ENSMUSG00000095183,ENSMUSG00000094760,ENSMUSG00000095749,ENSMUSG00000094657,ENSMUSG00000095645,ENSMUSG00000096765,ENSMUSG00000096170,ENSMUSG00000095453
Brain_47-m-18,2.197225,4.779123,2.890372,6.304449,3.688879,5.652489,2.397895,2.944439,6.040255,2.995732,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Brain_51-f-18,2.197225,4.219508,2.484907,5.529429,3.295837,5.298317,2.302585,0.693147,5.451038,2.302585,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Brain_20-f-21,2.302585,4.934474,2.995732,6.463029,4.043051,5.945421,2.564949,2.944439,6.180017,3.178054,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Brain_39-m-12,3.135494,4.897840,3.178054,6.386879,4.276666,5.981414,1.609438,2.890372,6.086775,2.564949,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Brain_4-m-3,2.484907,5.003946,3.178054,6.811244,3.806662,6.113682,3.367296,1.945910,6.359574,2.079442,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
data = brain_df_age.values

In [45]:
X = brain_log_transformed_data.values
y = data[:,-1]

In [47]:
loo = LeaveOneOut()

# Initialize lists to store actual and predicted values
y_true = []
y_pred = []
non_zero_coefs_list = []

# Loop over each train-test split
for train_index, test_index in loo.split(X):
    # Split the data
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Scale the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Define and train the model
    model = ElasticNet(alpha=0.01, l1_ra0.6000000000000001tio=1.0, max_iter=100000, random_state=seed)
    model.fit(X_train_scaled, y_train)
    #model.fit(X_train, y_train)

    # Get the number of non-zero coefficients
    non_zero_coefs = np.sum(model.coef_ != 0)
    non_zero_coefs_list.append(non_zero_coefs)
    
    # Make prediction on the test set
    y_test_pred = model.predict(X_test_scaled)
    #y_test_pred = model.predict(X_test)
    
    # Store the actual and predicted values
    y_true.append(y_test[0])
    y_pred.append(y_test_pred[0])

# Convert lists to numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Calculate the Pearson correlation coefficient
pearson_corr, _ = pearsonr(y_true, y_pred)

# Calculate the R-squared value
r_squared = r2_score(y_true, y_pred)

# Calculate the mean absolute error
mae = mean_absolute_error(y_true, y_pred)

print(f"Mean MAE: {mae:.3f}")
print(f"Pearson correlation: {pearson_corr:.3f}")
print(f"R-squared: {r_squared:.3f}")


# Print average and total non-zero coefficients across all LOO iterations
avg_non_zero_coefs = np.mean(non_zero_coefs_list)
print(f"Average number of non-zero coefficients: {avg_non_zero_coefs:.2f}")

Mean MAE: 2.190
Pearson correlation: 0.949
R-squared: 0.892
Average number of non-zero coefficients: 62.98


# GAT

In [48]:
gat_df_metadata = df_new_metadata[df_new_metadata["source.name"].str.startswith("GAT")]

gat_df_metadata.head(3)

,Unnamed: 0,Sample.name,title,source.name,organism,characteristics..age,characteristics..developmental.stage,characteristics..sex,molecule,description,processed.data.file,raw.file,BioSample,Instrument.Model
Sample.name.new,,,,,,,,,,,,,,
GAT_39-f-1,A10_384Bulk_Plate2_S10,A10_384Bulk_Plate2_S10,Tabula Muris Senis (bulk RNA seq),GAT_39,Mus musculus C57/BL6,1,months postnatal,f,total RNA,NaN,190214_A00111_0269_AHH3J3DSXX__190214_A00111_0...,SRR9127204,SAMN11854569,Illumina NovaSeq 6000
GAT_33-m-6,A15_384Bulk_Plate1_S15,A15_384Bulk_Plate1_S15,Tabula Muris Senis (bulk RNA seq),GAT_33,Mus musculus C57/BL6,6,months postnatal,m,total RNA,NaN,190214_A00111_0269_AHH3J3DSXX__190214_A00111_0...,SRR9127023,SAMN11854583,Illumina NovaSeq 6000
GAT_2-m-12,A4_384Bulk_Plate2_S4,A4_384Bulk_Plate2_S4,Tabula Muris Senis (bulk RNA seq),GAT_2,Mus musculus C57/BL6,12,months postnatal,m,total RNA,NaN,190214_A00111_0269_AHH3J3DSXX__190214_A00111_0...,SRR9127253,SAMN11854608,Illumina NovaSeq 6000


In [49]:
gat_df_counts = df.loc[gat_df_metadata.index.tolist(),]

gat_df_counts.head(3)

Gene,ENSMUSG00000109644,ENSMUSG00000007777,ENSMUSG00000043644,ENSMUSG00000042208,ENSMUSG00000020831,ENSMUSG00000107002,ENSMUSG00000058706,ENSMUSG00000099146,ENSMUSG00000028608,ENSMUSG00000097882,...,ENSMUSG00000096015,ENSMUSG00000096037,ENSMUSG00000095183,ENSMUSG00000094760,ENSMUSG00000095749,ENSMUSG00000094657,ENSMUSG00000095645,ENSMUSG00000096765,ENSMUSG00000096170,ENSMUSG00000095453
GAT_39-f-1,29,269,40,570,113,488,87,0,532,44,...,0,0,0,0,0,0,0,0,0,0
GAT_33-m-6,15,155,60,391,50,320,86,0,408,25,...,0,0,0,0,0,0,0,0,0,0
GAT_2-m-12,81,218,60,765,75,504,64,9,499,27,...,0,0,0,0,0,0,0,0,0,0


In [50]:
gat_df_age = gat_df_counts.copy()

In [51]:
gat_df_age["age"] = gat_df_metadata["characteristics..age"].tolist()

In [52]:
gat_df_age.head(5)

Gene,ENSMUSG00000109644,ENSMUSG00000007777,ENSMUSG00000043644,ENSMUSG00000042208,ENSMUSG00000020831,ENSMUSG00000107002,ENSMUSG00000058706,ENSMUSG00000099146,ENSMUSG00000028608,ENSMUSG00000097882,...,ENSMUSG00000096037,ENSMUSG00000095183,ENSMUSG00000094760,ENSMUSG00000095749,ENSMUSG00000094657,ENSMUSG00000095645,ENSMUSG00000096765,ENSMUSG00000096170,ENSMUSG00000095453,age
GAT_39-f-1,29,269,40,570,113,488,87,0,532,44,...,0,0,0,0,0,0,0,0,0,1
GAT_33-m-6,15,155,60,391,50,320,86,0,408,25,...,0,0,0,0,0,0,0,0,0,6
GAT_2-m-12,81,218,60,765,75,504,64,9,499,27,...,0,0,0,0,0,0,0,0,0,12
GAT_8-m-1,21,214,22,678,87,418,113,5,602,11,...,0,0,0,0,0,0,0,0,0,1
GAT_15-m-12,22,216,46,442,55,374,38,1,390,19,...,0,0,0,0,0,0,0,0,0,12


In [53]:
gat_df_age["age"] = pd.to_numeric(gat_df_metadata["characteristics..age"].tolist(), errors="coerce")

In [54]:
gat_log_transformed_data = np.log1p(gat_df_counts)

In [55]:
gat_data = gat_df_age.values

In [56]:
X = gat_log_transformed_data.values
y = gat_data[:,-1]

In [57]:
loo = LeaveOneOut()

# Initialize lists to store actual and predicted values
y_true = []
y_pred = []

# Loop over each train-test split
for train_index, test_index in loo.split(X):
    # Split the data
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Scale the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Define and train the model
    model = ElasticNet(alpha=0.01, l1_ratio=0.6000000000000001, max_iter=100000, random_state=seed)
    model.fit(X_train_scaled, y_train)
    #model.fit(X_train, y_train)
    
    # Make prediction on the test set
    y_test_pred = model.predict(X_test_scaled)
    #y_test_pred = model.predict(X_test)
    
    # Store the actual and predicted values
    y_true.append(y_test[0])
    y_pred.append(y_test_pred[0])

# Convert lists to numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Calculate the Pearson correlation coefficient
pearson_corr, _ = pearsonr(y_true, y_pred)

# Calculate the R-squared value
r_squared = r2_score(y_true, y_pred)

# Calculate the mean absolute error
mae = mean_absolute_error(y_true, y_pred)

print(f"Mean MAE: {mae:.3f}")
print(f"Pearson correlation: {pearson_corr:.3f}")
print(f"R-squared: {r_squared:.3f}")

Mean MAE: 2.040
Pearson correlation: 0.955
R-squared: 0.905


# Heart

In [58]:
heart_df_metadata = df_new_metadata[df_new_metadata["source.name"].str.startswith("Heart")]

heart_df_metadata.head(3)

,Unnamed: 0,Sample.name,title,source.name,organism,characteristics..age,characteristics..developmental.stage,characteristics..sex,molecule,description,processed.data.file,raw.file,BioSample,Instrument.Model
Sample.name.new,,,,,,,,,,,,,,
Heart_8-m-1,A19_384Bulk_Plate2_S19,A19_384Bulk_Plate2_S19,Tabula Muris Senis (bulk RNA seq),Heart_8,Mus musculus C57/BL6,1,months postnatal,m,total RNA,NaN,190214_A00111_0269_AHH3J3DSXX__190214_A00111_0...,SRR9127070,SAMN11854592,Illumina NovaSeq 6000
Heart_47-m-9,B20_384Bulk_Plate2_S44,B20_384Bulk_Plate2_S44,Tabula Muris Senis (bulk RNA seq),Heart_47,Mus musculus C57/BL6,9,months postnatal,m,total RNA,NaN,190214_A00111_0269_AHH3J3DSXX__190214_A00111_0...,SRR9127134,SAMN11854657,Illumina NovaSeq 6000
Heart_42-m-1,B22_384Bulk_Plate2_S46,B22_384Bulk_Plate2_S46,Tabula Muris Senis (bulk RNA seq),Heart_42,Mus musculus C57/BL6,1,months postnatal,m,total RNA,NaN,190214_A00111_0269_AHH3J3DSXX__190214_A00111_0...,SRR9127130,SAMN11854661,Illumina NovaSeq 6000


In [59]:
heart_df_counts = df.loc[heart_df_metadata.index.tolist(),]

heart_df_counts.head(3)

Gene,ENSMUSG00000109644,ENSMUSG00000007777,ENSMUSG00000043644,ENSMUSG00000042208,ENSMUSG00000020831,ENSMUSG00000107002,ENSMUSG00000058706,ENSMUSG00000099146,ENSMUSG00000028608,ENSMUSG00000097882,...,ENSMUSG00000096015,ENSMUSG00000096037,ENSMUSG00000095183,ENSMUSG00000094760,ENSMUSG00000095749,ENSMUSG00000094657,ENSMUSG00000095645,ENSMUSG00000096765,ENSMUSG00000096170,ENSMUSG00000095453
Heart_8-m-1,2,117,35,154,25,436,15,1,301,19,...,0,0,0,0,0,0,0,0,0,0
Heart_47-m-9,11,141,44,289,16,514,16,19,343,16,...,0,0,0,0,0,0,0,0,0,0
Heart_42-m-1,13,174,48,234,42,533,26,11,353,36,...,0,0,0,0,0,0,0,0,0,0


In [60]:
heart_df_age = heart_df_counts.copy()

In [61]:
heart_df_age["age"] = heart_df_metadata["characteristics..age"].tolist()

In [62]:
heart_df_age.head(5)

Gene,ENSMUSG00000109644,ENSMUSG00000007777,ENSMUSG00000043644,ENSMUSG00000042208,ENSMUSG00000020831,ENSMUSG00000107002,ENSMUSG00000058706,ENSMUSG00000099146,ENSMUSG00000028608,ENSMUSG00000097882,...,ENSMUSG00000096037,ENSMUSG00000095183,ENSMUSG00000094760,ENSMUSG00000095749,ENSMUSG00000094657,ENSMUSG00000095645,ENSMUSG00000096765,ENSMUSG00000096170,ENSMUSG00000095453,age
Heart_8-m-1,2,117,35,154,25,436,15,1,301,19,...,0,0,0,0,0,0,0,0,0,1
Heart_47-m-9,11,141,44,289,16,514,16,19,343,16,...,0,0,0,0,0,0,0,0,0,9
Heart_42-m-1,13,174,48,234,42,533,26,11,353,36,...,0,0,0,0,0,0,0,0,0,1
Heart_38-f-21,11,135,63,267,31,524,17,40,533,30,...,0,0,0,0,0,0,0,0,0,21
Heart_15-m-27,2,128,44,152,13,490,24,1,302,9,...,0,0,0,0,0,0,0,0,0,27


In [63]:
heart_df_age["age"] = pd.to_numeric(heart_df_metadata["characteristics..age"].tolist(), errors="coerce")

In [64]:
heart_df_age.head(3)

Gene,ENSMUSG00000109644,ENSMUSG00000007777,ENSMUSG00000043644,ENSMUSG00000042208,ENSMUSG00000020831,ENSMUSG00000107002,ENSMUSG00000058706,ENSMUSG00000099146,ENSMUSG00000028608,ENSMUSG00000097882,...,ENSMUSG00000096037,ENSMUSG00000095183,ENSMUSG00000094760,ENSMUSG00000095749,ENSMUSG00000094657,ENSMUSG00000095645,ENSMUSG00000096765,ENSMUSG00000096170,ENSMUSG00000095453,age
Heart_8-m-1,2,117,35,154,25,436,15,1,301,19,...,0,0,0,0,0,0,0,0,0,1
Heart_47-m-9,11,141,44,289,16,514,16,19,343,16,...,0,0,0,0,0,0,0,0,0,9
Heart_42-m-1,13,174,48,234,42,533,26,11,353,36,...,0,0,0,0,0,0,0,0,0,1


In [65]:
heart_log_transformed_data = np.log1p(heart_df_counts)

In [66]:
heart_data = heart_df_age.values

In [67]:
X = heart_log_transformed_data.values
y = heart_data[:,-1]

In [ ]:
## Define the parameter grid
param_grid = {
    'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1.0, 10.0, 100.0],
    'l1_ratio': np.arange(0, 1.1, 0.1)
}

# Define the model
model = ElasticNet(max_iter=10000)

# Define Leave-One-Out cross-validation
loo = LeaveOneOut()

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=loo, scoring='neg_mean_absolute_error', n_jobs=-1)

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Fit the GridSearchCV
grid_search.fit(X_scaled, y)

# Get the best parameters
best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

# Train the model with the best parameters using Leave-One-Out cross-validation
y_true = []
y_pred = []

for train_index, test_index in loo.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    best_model = ElasticNet(alpha=best_params['alpha'], l1_ratio=best_params['l1_ratio'], max_iter=10000, random_state=seed)
    best_model.fit(X_train_scaled, y_train)
    
    y_test_pred = best_model.predict(X_test_scaled)
    
    y_true.append(y_test[0])
    y_pred.append(y_test_pred[0])

# Convert lists to numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Calculate the Pearson correlation coefficient
pearson_corr, _ = pearsonr(y_true, y_pred)

# Calculate the R-squared value
r_squared = r2_score(y_true, y_pred)

# Calculate the mean absolute error
mae = mean_absolute_error(y_true, y_pred)

print(f"Mean MAE: {mae:.3f}")
print(f"Pearson correlation: {pearson_corr:.3f}")
print(f"R-squared: {r_squared:.3f}")

In [69]:
loo = LeaveOneOut()

# Initialize lists to store actual and predicted values
y_true = []
y_pred = []

# Loop over each train-test split
for train_index, test_index in loo.split(X):
    # Split the data
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Scale the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Define and train the model
    model = ElasticNet(alpha=0.01, l1_ratio=1.0, max_iter=100000, random_state=seed)
    model.fit(X_train_scaled, y_train)
    #model.fit(X_train, y_train)
    
    # Make prediction on the test set
    y_test_pred = model.predict(X_test_scaled)
    #y_test_pred = model.predict(X_test)
    
    # Store the actual and predicted values
    y_true.append(y_test[0])
    y_pred.append(y_test_pred[0])

# Convert lists to numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Calculate the Pearson correlation coefficient
pearson_corr, _ = pearsonr(y_true, y_pred)

# Calculate the R-squared value
r_squared = r2_score(y_true, y_pred)

# Calculate the mean absolute error
mae = mean_absolute_error(y_true, y_pred)

print(f"Mean MAE: {mae:.3f}")
print(f"Pearson correlation: {pearson_corr:.3f}")
print(f"R-squared: {r_squared:.3f}")

Mean MAE: 4.829
Pearson correlation: 0.578
R-squared: 0.267


# Kidney

In [70]:
kidney_df_metadata = df_new_metadata[df_new_metadata["source.name"].str.startswith("Kidney")]

kidney_df_metadata.head(3)

,Unnamed: 0,Sample.name,title,source.name,organism,characteristics..age,characteristics..developmental.stage,characteristics..sex,molecule,description,processed.data.file,raw.file,BioSample,Instrument.Model
Sample.name.new,,,,,,,,,,,,,,
Kidney_45-f-9,A5_384Bulk_Plate2_S5,A5_384Bulk_Plate2_S5,Tabula Muris Senis (bulk RNA seq),Kidney_45,Mus musculus C57/BL6,9,months postnatal,f,total RNA,NaN,190214_A00111_0269_AHH3J3DSXX__190214_A00111_0...,SRR9127254,SAMN11854611,Illumina NovaSeq 6000
Kidney_38-m-21,B14_384Bulk_Plate1_S38,B14_384Bulk_Plate1_S38,Tabula Muris Senis (bulk RNA seq),Kidney_38,Mus musculus C57/BL6,21,months postnatal,m,total RNA,NaN,190214_A00111_0269_AHH3J3DSXX__190214_A00111_0...,SRR9126845,SAMN11854640,Illumina NovaSeq 6000
Kidney_44-m-1,B17_384Bulk_Plate2_S41,B17_384Bulk_Plate2_S41,Tabula Muris Senis (bulk RNA seq),Kidney_44,Mus musculus C57/BL6,1,months postnatal,m,total RNA,NaN,190214_A00111_0269_AHH3J3DSXX__190214_A00111_0...,SRR9127582,SAMN11854648,Illumina NovaSeq 6000


In [71]:
kidney_df_counts = df.loc[kidney_df_metadata.index.tolist(),]

kidney_df_counts.head(3)

Gene,ENSMUSG00000109644,ENSMUSG00000007777,ENSMUSG00000043644,ENSMUSG00000042208,ENSMUSG00000020831,ENSMUSG00000107002,ENSMUSG00000058706,ENSMUSG00000099146,ENSMUSG00000028608,ENSMUSG00000097882,...,ENSMUSG00000096015,ENSMUSG00000096037,ENSMUSG00000095183,ENSMUSG00000094760,ENSMUSG00000095749,ENSMUSG00000094657,ENSMUSG00000095645,ENSMUSG00000096765,ENSMUSG00000096170,ENSMUSG00000095453
Kidney_45-f-9,1709,163,24,250,41,494,46,13,322,23,...,0,0,0,0,0,0,0,0,0,0
Kidney_38-m-21,1174,182,32,99,33,360,47,52,187,10,...,0,0,0,0,0,0,0,0,0,0
Kidney_44-m-1,1709,396,68,341,53,714,77,149,390,87,...,0,0,0,0,0,0,0,0,0,0


In [72]:
kidney_df_age = kidney_df_counts.copy()

In [73]:
kidney_df_age["age"] = kidney_df_metadata["characteristics..age"].tolist()

In [74]:
kidney_df_age["age"] = pd.to_numeric(kidney_df_metadata["characteristics..age"].tolist(), errors="coerce")

In [75]:
kidney_df_age.head(3)

Gene,ENSMUSG00000109644,ENSMUSG00000007777,ENSMUSG00000043644,ENSMUSG00000042208,ENSMUSG00000020831,ENSMUSG00000107002,ENSMUSG00000058706,ENSMUSG00000099146,ENSMUSG00000028608,ENSMUSG00000097882,...,ENSMUSG00000096037,ENSMUSG00000095183,ENSMUSG00000094760,ENSMUSG00000095749,ENSMUSG00000094657,ENSMUSG00000095645,ENSMUSG00000096765,ENSMUSG00000096170,ENSMUSG00000095453,age
Kidney_45-f-9,1709,163,24,250,41,494,46,13,322,23,...,0,0,0,0,0,0,0,0,0,9
Kidney_38-m-21,1174,182,32,99,33,360,47,52,187,10,...,0,0,0,0,0,0,0,0,0,21
Kidney_44-m-1,1709,396,68,341,53,714,77,149,390,87,...,0,0,0,0,0,0,0,0,0,1


In [76]:
kidney_log_transformed_data = np.log1p(kidney_df_counts)

In [77]:
kidney_data = kidney_df_age.values

In [78]:
X = kidney_log_transformed_data.values
y = kidney_data[:,-1]

In [ ]:
## Define the parameter grid
param_grid = {
    'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1.0, 10.0, 100.0],
    'l1_ratio': np.arange(0, 1.1, 0.1)
}

# Define the model
model = ElasticNet(max_iter=10000, random_state=seed)

# Define Leave-One-Out cross-validation
loo = LeaveOneOut()

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=loo, scoring='neg_mean_absolute_error', n_jobs=-1)

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Fit the GridSearchCV
grid_search.fit(X_scaled, y)

# Get the best parameters
best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

# Train the model with the best parameters using Leave-One-Out cross-validation
y_true = []
y_pred = []

for train_index, test_index in loo.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    best_model = ElasticNet(alpha=best_params['alpha'], l1_ratio=best_params['l1_ratio'], max_iter=10000, random_state=seed)
    best_model.fit(X_train_scaled, y_train)
    
    y_test_pred = best_model.predict(X_test_scaled)
    
    y_true.append(y_test[0])
    y_pred.append(y_test_pred[0])

# Convert lists to numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Calculate the Pearson correlation coefficient
pearson_corr, _ = pearsonr(y_true, y_pred)

# Calculate the R-squared value
r_squared = r2_score(y_true, y_pred)

# Calculate the mean absolute error
mae = mean_absolute_error(y_true, y_pred)

print(f"Mean MAE: {mae:.3f}")
print(f"Pearson correlation: {pearson_corr:.3f}")
print(f"R-squared: {r_squared:.3f}")

In [80]:
loo = LeaveOneOut()

# Initialize lists to store actual and predicted values
y_true = []
y_pred = []

# Loop over each train-test split
for train_index, test_index in loo.split(X):
    # Split the data
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Scale the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Define and train the model
    model = ElasticNet(alpha=0.001, l1_ratio=0.8, max_iter=100000, random_state=seed)
    model.fit(X_train_scaled, y_train)
    #model.fit(X_train, y_train)
    
    # Make prediction on the test set
    y_test_pred = model.predict(X_test_scaled)
    #y_test_pred = model.predict(X_test)
    
    # Store the actual and predicted values
    y_true.append(y_test[0])
    y_pred.append(y_test_pred[0])

# Convert lists to numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Calculate the Pearson correlation coefficient
pearson_corr, _ = pearsonr(y_true, y_pred)

# Calculate the R-squared value
r_squared = r2_score(y_true, y_pred)

# Calculate the mean absolute error
mae = mean_absolute_error(y_true, y_pred)

print(f"Mean MAE: {mae:.3f}")
print(f"Pearson correlation: {pearson_corr:.3f}")
print(f"R-squared: {r_squared:.3f}")

Mean MAE: 2.338
Pearson correlation: 0.943
R-squared: 0.867


# Limb Muscle

In [82]:
limb_muscle_df_metadata = df_new_metadata[df_new_metadata["source.name"].str.startswith("Limb")]

limb_muscle_df_metadata.head(3)

,Unnamed: 0,Sample.name,title,source.name,organism,characteristics..age,characteristics..developmental.stage,characteristics..sex,molecule,description,processed.data.file,raw.file,BioSample,Instrument.Model
Sample.name.new,,,,,,,,,,,,,,
Limb_Muscle_11-m-3,A11_384Bulk_Plate1_S11,A11_384Bulk_Plate1_S11,Tabula Muris Senis (bulk RNA seq),Limb_Muscle_11,Mus musculus C57/BL6,3,months postnatal,m,total RNA,NaN,190214_A00111_0269_AHH3J3DSXX__190214_A00111_0...,SRR9127373,SAMN11854571,Illumina NovaSeq 6000
Limb_Muscle_38-m-9,A12_384Bulk_Plate1_S12,A12_384Bulk_Plate1_S12,Tabula Muris Senis (bulk RNA seq),Limb_Muscle_38,Mus musculus C57/BL6,9,months postnatal,m,total RNA,NaN,190214_A00111_0269_AHH3J3DSXX__190214_A00111_0...,SRR9127454,SAMN11854574,Illumina NovaSeq 6000
Limb_Muscle_18-m-6,A18_384Bulk_Plate2_S18,A18_384Bulk_Plate2_S18,Tabula Muris Senis (bulk RNA seq),Limb_Muscle_18,Mus musculus C57/BL6,6,months postnatal,m,total RNA,NaN,190214_A00111_0269_AHH3J3DSXX__190214_A00111_0...,SRR9127072,SAMN11854590,Illumina NovaSeq 6000


In [83]:
limb_df_counts = df.loc[limb_muscle_df_metadata.index.tolist(),]

limb_df_counts.head(3)

Gene,ENSMUSG00000109644,ENSMUSG00000007777,ENSMUSG00000043644,ENSMUSG00000042208,ENSMUSG00000020831,ENSMUSG00000107002,ENSMUSG00000058706,ENSMUSG00000099146,ENSMUSG00000028608,ENSMUSG00000097882,...,ENSMUSG00000096015,ENSMUSG00000096037,ENSMUSG00000095183,ENSMUSG00000094760,ENSMUSG00000095749,ENSMUSG00000094657,ENSMUSG00000095645,ENSMUSG00000096765,ENSMUSG00000096170,ENSMUSG00000095453
Limb_Muscle_11-m-3,6,209,148,259,19,635,24,0,222,30,...,0,0,0,0,0,0,0,0,0,0
Limb_Muscle_38-m-9,4,96,88,195,9,475,25,3,190,14,...,0,0,0,0,0,0,0,0,0,0
Limb_Muscle_18-m-6,1,187,110,312,31,761,16,3,253,25,...,0,0,0,0,0,0,0,0,0,0


In [84]:
limb_df_age = limb_df_counts.copy()

In [85]:
limb_df_age["age"] = limb_muscle_df_metadata["characteristics..age"].tolist()

In [86]:
limb_df_age["age"] = pd.to_numeric(limb_muscle_df_metadata["characteristics..age"].tolist(), errors="coerce")

In [87]:
limb_df_age.head(3)

Gene,ENSMUSG00000109644,ENSMUSG00000007777,ENSMUSG00000043644,ENSMUSG00000042208,ENSMUSG00000020831,ENSMUSG00000107002,ENSMUSG00000058706,ENSMUSG00000099146,ENSMUSG00000028608,ENSMUSG00000097882,...,ENSMUSG00000096037,ENSMUSG00000095183,ENSMUSG00000094760,ENSMUSG00000095749,ENSMUSG00000094657,ENSMUSG00000095645,ENSMUSG00000096765,ENSMUSG00000096170,ENSMUSG00000095453,age
Limb_Muscle_11-m-3,6,209,148,259,19,635,24,0,222,30,...,0,0,0,0,0,0,0,0,0,3
Limb_Muscle_38-m-9,4,96,88,195,9,475,25,3,190,14,...,0,0,0,0,0,0,0,0,0,9
Limb_Muscle_18-m-6,1,187,110,312,31,761,16,3,253,25,...,0,0,0,0,0,0,0,0,0,6


In [88]:
limb_log_transformed_data = np.log1p(limb_df_counts)

In [89]:
limb_data = limb_df_age.values

In [90]:
X = limb_log_transformed_data.values
y = limb_data[:,-1]

In [ ]:
## Define the parameter grid
param_grid = {
    'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1.0, 10.0, 100.0],
    'l1_ratio': np.arange(0, 1.1, 0.1)
}

# Define the model
model = ElasticNet(max_iter=10000, random_state=seed)

# Define Leave-One-Out cross-validation
loo = LeaveOneOut()

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=loo, scoring='neg_mean_absolute_error', n_jobs=-1)

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Fit the GridSearchCV
grid_search.fit(X_scaled, y)

# Get the best parameters
best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

# Train the model with the best parameters using Leave-One-Out cross-validation
y_true = []
y_pred = []

for train_index, test_index in loo.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    best_model = ElasticNet(alpha=best_params['alpha'], l1_ratio=best_params['l1_ratio'], max_iter=10000, random_state=seed)
    best_model.fit(X_train_scaled, y_train)
    
    y_test_pred = best_model.predict(X_test_scaled)
    
    y_true.append(y_test[0])
    y_pred.append(y_test_pred[0])

# Convert lists to numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Calculate the Pearson correlation coefficient
pearson_corr, _ = pearsonr(y_true, y_pred)

# Calculate the R-squared value
r_squared = r2_score(y_true, y_pred)

# Calculate the mean absolute error
mae = mean_absolute_error(y_true, y_pred)

print(f"Mean MAE: {mae:.3f}")
print(f"Pearson correlation: {pearson_corr:.3f}")
print(f"R-squared: {r_squared:.3f}")

In [92]:
loo = LeaveOneOut()

# Initialize lists to store actual and predicted values
y_true = []
y_pred = []

# Loop over each train-test split
for train_index, test_index in loo.split(X):
    # Split the data
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Scale the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Define and train the model
    model = ElasticNet(alpha=0.01, l1_ratio=0.9, max_iter=100000, random_state=seed)
    model.fit(X_train_scaled, y_train)
    #model.fit(X_train, y_train)
    
    # Make prediction on the test set
    y_test_pred = model.predict(X_test_scaled)
    #y_test_pred = model.predict(X_test)
    
    # Store the actual and predicted values
    y_true.append(y_test[0])
    y_pred.append(y_test_pred[0])

# Convert lists to numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Calculate the Pearson correlation coefficient
pearson_corr, _ = pearsonr(y_true, y_pred)

# Calculate the R-squared value
r_squared = r2_score(y_true, y_pred)

# Calculate the mean absolute error
mae = mean_absolute_error(y_true, y_pred)

print(f"Mean MAE: {mae:.3f}")
print(f"Pearson correlation: {pearson_corr:.3f}")
print(f"R-squared: {r_squared:.3f}")

Mean MAE: 2.468
Pearson correlation: 0.930
R-squared: 0.850


# Liver

In [93]:
liver_df_metadata = df_new_metadata[df_new_metadata["source.name"].str.startswith("Liver")]

liver_df_metadata.head(3)

,Unnamed: 0,Sample.name,title,source.name,organism,characteristics..age,characteristics..developmental.stage,characteristics..sex,molecule,description,processed.data.file,raw.file,BioSample,Instrument.Model
Sample.name.new,,,,,,,,,,,,,,
Liver_56-m-6,A12_384Bulk_Plate2_S12,A12_384Bulk_Plate2_S12,Tabula Muris Senis (bulk RNA seq),Liver_56,Mus musculus C57/BL6,6,months postnatal,m,total RNA,NaN,190214_A00111_0269_AHH3J3DSXX__190214_A00111_0...,SRR9126831,SAMN11854575,Illumina NovaSeq 6000
Liver_46-m-6,A21_384Bulk_Plate2_S21,A21_384Bulk_Plate2_S21,Tabula Muris Senis (bulk RNA seq),Liver_46,Mus musculus C57/BL6,6,months postnatal,m,total RNA,NaN,190214_A00111_0269_AHH3J3DSXX__190214_A00111_0...,SRR9127503,SAMN11854599,Illumina NovaSeq 6000
Liver_37-m-1,A22_384Bulk_Plate2_S22,A22_384Bulk_Plate2_S22,Tabula Muris Senis (bulk RNA seq),Liver_37,Mus musculus C57/BL6,1,months postnatal,m,total RNA,NaN,190214_A00111_0269_AHH3J3DSXX__190214_A00111_0...,SRR9127505,SAMN11854601,Illumina NovaSeq 6000


In [94]:
liver_df_counts = df.loc[liver_df_metadata.index.tolist(),]

liver_df_counts.head(3)

Gene,ENSMUSG00000109644,ENSMUSG00000007777,ENSMUSG00000043644,ENSMUSG00000042208,ENSMUSG00000020831,ENSMUSG00000107002,ENSMUSG00000058706,ENSMUSG00000099146,ENSMUSG00000028608,ENSMUSG00000097882,...,ENSMUSG00000096015,ENSMUSG00000096037,ENSMUSG00000095183,ENSMUSG00000094760,ENSMUSG00000095749,ENSMUSG00000094657,ENSMUSG00000095645,ENSMUSG00000096765,ENSMUSG00000096170,ENSMUSG00000095453
Liver_56-m-6,2494,214,12,327,21,346,144,630,193,1,...,0,0,0,0,0,0,0,0,0,0
Liver_46-m-6,1746,190,1,102,16,184,48,293,161,10,...,0,0,0,0,0,0,0,0,0,0
Liver_37-m-1,1255,178,4,191,9,217,61,364,129,3,...,0,0,0,0,0,0,0,0,0,0


In [95]:
liver_df_age = liver_df_counts.copy()

In [96]:
liver_df_age["age"] = liver_df_metadata["characteristics..age"].tolist()

In [97]:
liver_df_age.head(5)

Gene,ENSMUSG00000109644,ENSMUSG00000007777,ENSMUSG00000043644,ENSMUSG00000042208,ENSMUSG00000020831,ENSMUSG00000107002,ENSMUSG00000058706,ENSMUSG00000099146,ENSMUSG00000028608,ENSMUSG00000097882,...,ENSMUSG00000096037,ENSMUSG00000095183,ENSMUSG00000094760,ENSMUSG00000095749,ENSMUSG00000094657,ENSMUSG00000095645,ENSMUSG00000096765,ENSMUSG00000096170,ENSMUSG00000095453,age
Liver_56-m-6,2494,214,12,327,21,346,144,630,193,1,...,0,0,0,0,0,0,0,0,0,6
Liver_46-m-6,1746,190,1,102,16,184,48,293,161,10,...,0,0,0,0,0,0,0,0,0,6
Liver_37-m-1,1255,178,4,191,9,217,61,364,129,3,...,0,0,0,0,0,0,0,0,0,1
Liver_8-f-3,2391,288,6,347,36,344,98,836,230,1,...,0,0,0,0,0,0,0,0,0,3
Liver_14-m-3,1599,191,0,178,11,256,64,528,171,1,...,0,0,0,0,0,0,0,0,0,3


In [98]:
liver_df_age["age"] = pd.to_numeric(liver_df_metadata["characteristics..age"].tolist(), errors="coerce")

In [99]:
liver_log_transformed_data = np.log1p(liver_df_counts)

In [100]:
liver_data = liver_df_age.values

In [101]:
X = liver_log_transformed_data.values
y = liver_data[:,-1]

In [ ]:
## Define the parameter grid
param_grid = {
    'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1.0, 10.0, 100.0],
    'l1_ratio': np.arange(0, 1.1, 0.1)
}

# Define the model
model = ElasticNet(max_iter=10000, random_state=seed)

# Define Leave-One-Out cross-validation
loo = LeaveOneOut()

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=loo, scoring='neg_mean_absolute_error', n_jobs=-1)

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Fit the GridSearchCV
grid_search.fit(X_scaled, y)

# Get the best parameters
best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

# Train the model with the best parameters using Leave-One-Out cross-validation
y_true = []
y_pred = []

for train_index, test_index in loo.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    best_model = ElasticNet(alpha=best_params['alpha'], l1_ratio=best_params['l1_ratio'], max_iter=10000, random_state=seed)
    best_model.fit(X_train_scaled, y_train)
    
    y_test_pred = best_model.predict(X_test_scaled)
    
    y_true.append(y_test[0])
    y_pred.append(y_test_pred[0])

# Convert lists to numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Calculate the Pearson correlation coefficient
pearson_corr, _ = pearsonr(y_true, y_pred)

# Calculate the R-squared value
r_squared = r2_score(y_true, y_pred)

# Calculate the mean absolute error
mae = mean_absolute_error(y_true, y_pred)

print(f"Mean MAE: {mae:.3f}")
print(f"Pearson correlation: {pearson_corr:.3f}")
print(f"R-squared: {r_squared:.3f}")

In [103]:
loo = LeaveOneOut()

# Initialize lists to store actual and predicted values
y_true = []
y_pred = []

# Loop over each train-test split
for train_index, test_index in loo.split(X):
    # Split the data
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Scale the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Define and train the model
    model = ElasticNet(alpha=0.01, l1_ratio=1.0, max_iter=100000, random_state=seed)
    model.fit(X_train_scaled, y_train)
    #model.fit(X_train, y_train)
    
    # Make prediction on the test set
    y_test_pred = model.predict(X_test_scaled)
    #y_test_pred = model.predict(X_test)
    
    # Store the actual and predicted values
    y_true.append(y_test[0])
    y_pred.append(y_test_pred[0])

# Convert lists to numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Calculate the Pearson correlation coefficient
pearson_corr, _ = pearsonr(y_true, y_pred)

# Calculate the R-squared value
r_squared = r2_score(y_true, y_pred)

# Calculate the mean absolute error
mae = mean_absolute_error(y_true, y_pred)

print(f"Mean MAE: {mae:.3f}")
print(f"Pearson correlation: {pearson_corr:.3f}")
print(f"R-squared: {r_squared:.3f}")

Mean MAE: 2.121
Pearson correlation: 0.944
R-squared: 0.884


# Lung

In [104]:
lung_df_metadata = df_new_metadata[df_new_metadata["source.name"].str.startswith("Lung")]

lung_df_metadata.head(3)

,Unnamed: 0,Sample.name,title,source.name,organism,characteristics..age,characteristics..developmental.stage,characteristics..sex,molecule,description,processed.data.file,raw.file,BioSample,Instrument.Model
Sample.name.new,,,,,,,,,,,,,,
Lung_6-f-12,A10_384Bulk_Plate3_S10,A10_384Bulk_Plate3_S10,Tabula Muris Senis (bulk RNA seq),Lung_6,Mus musculus C57/BL6,12,months postnatal,f,total RNA,NaN,190214_A00111_0269_AHH3J3DSXX__190214_A00111_0...,SRR9127371,SAMN11854570,Illumina NovaSeq 6000
Lung_17-f-12,A12_384Bulk_Plate3_S12,A12_384Bulk_Plate3_S12,Tabula Muris Senis (bulk RNA seq),Lung_17,Mus musculus C57/BL6,12,months postnatal,f,total RNA,NaN,190214_A00111_0269_AHH3J3DSXX__190214_A00111_0...,SRR9127028,SAMN11854576,Illumina NovaSeq 6000
Lung_7-m-1,A16_384Bulk_Plate1_S16,A16_384Bulk_Plate1_S16,Tabula Muris Senis (bulk RNA seq),Lung_7,Mus musculus C57/BL6,1,months postnatal,m,total RNA,NaN,190214_A00111_0269_AHH3J3DSXX__190214_A00111_0...,SRR9127021,SAMN11854585,Illumina NovaSeq 6000


In [105]:
lung_df_counts = df.loc[lung_df_metadata.index.tolist(),]

lung_df_counts.head(3)

Gene,ENSMUSG00000109644,ENSMUSG00000007777,ENSMUSG00000043644,ENSMUSG00000042208,ENSMUSG00000020831,ENSMUSG00000107002,ENSMUSG00000058706,ENSMUSG00000099146,ENSMUSG00000028608,ENSMUSG00000097882,...,ENSMUSG00000096015,ENSMUSG00000096037,ENSMUSG00000095183,ENSMUSG00000094760,ENSMUSG00000095749,ENSMUSG00000094657,ENSMUSG00000095645,ENSMUSG00000096765,ENSMUSG00000096170,ENSMUSG00000095453
Lung_6-f-12,5,117,21,291,40,204,40,9,471,13,...,0,0,0,0,0,0,0,0,0,0
Lung_17-f-12,7,91,42,230,33,183,59,8,510,29,...,0,0,0,0,0,0,0,0,0,0
Lung_7-m-1,5,147,31,352,66,234,86,8,548,15,...,0,0,0,0,0,0,0,0,0,0


In [106]:
lung_df_age = lung_df_counts.copy()

In [107]:
lung_df_age["age"] = lung_df_metadata["characteristics..age"].tolist()

In [108]:
lung_df_age.head(5)

Gene,ENSMUSG00000109644,ENSMUSG00000007777,ENSMUSG00000043644,ENSMUSG00000042208,ENSMUSG00000020831,ENSMUSG00000107002,ENSMUSG00000058706,ENSMUSG00000099146,ENSMUSG00000028608,ENSMUSG00000097882,...,ENSMUSG00000096037,ENSMUSG00000095183,ENSMUSG00000094760,ENSMUSG00000095749,ENSMUSG00000094657,ENSMUSG00000095645,ENSMUSG00000096765,ENSMUSG00000096170,ENSMUSG00000095453,age
Lung_6-f-12,5,117,21,291,40,204,40,9,471,13,...,0,0,0,0,0,0,0,0,0,12
Lung_17-f-12,7,91,42,230,33,183,59,8,510,29,...,0,0,0,0,0,0,0,0,0,12
Lung_7-m-1,5,147,31,352,66,234,86,8,548,15,...,0,0,0,0,0,0,0,0,0,1
Lung_39-m-3,11,48,36,249,65,154,39,9,390,17,...,0,0,0,0,0,0,0,0,0,3
Lung_11-m-18,13,185,27,385,91,315,93,20,676,14,...,0,0,0,0,0,0,0,0,0,18


In [109]:
lung_df_age["age"] = pd.to_numeric(lung_df_metadata["characteristics..age"].tolist(), errors="coerce")

In [110]:
lung_df_age.head(3)

Gene,ENSMUSG00000109644,ENSMUSG00000007777,ENSMUSG00000043644,ENSMUSG00000042208,ENSMUSG00000020831,ENSMUSG00000107002,ENSMUSG00000058706,ENSMUSG00000099146,ENSMUSG00000028608,ENSMUSG00000097882,...,ENSMUSG00000096037,ENSMUSG00000095183,ENSMUSG00000094760,ENSMUSG00000095749,ENSMUSG00000094657,ENSMUSG00000095645,ENSMUSG00000096765,ENSMUSG00000096170,ENSMUSG00000095453,age
Lung_6-f-12,5,117,21,291,40,204,40,9,471,13,...,0,0,0,0,0,0,0,0,0,12
Lung_17-f-12,7,91,42,230,33,183,59,8,510,29,...,0,0,0,0,0,0,0,0,0,12
Lung_7-m-1,5,147,31,352,66,234,86,8,548,15,...,0,0,0,0,0,0,0,0,0,1


In [111]:
lung_log_transformed_data = np.log1p(lung_df_counts)

In [112]:
lung_data = lung_df_age.values

In [113]:
X = lung_log_transformed_data.values
y = lung_data[:,-1]

In [ ]:
## Define the parameter grid
param_grid = {
    'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1.0, 10.0, 100.0],
    'l1_ratio': np.arange(0, 1.1, 0.1)
}

# Define the model
model = ElasticNet(max_iter=10000, random_state=seed)

# Define Leave-One-Out cross-validation
loo = LeaveOneOut()

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=loo, scoring='neg_mean_absolute_error', n_jobs=-1)

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Fit the GridSearchCV
grid_search.fit(X_scaled, y)

# Get the best parameters
best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

# Train the model with the best parameters using Leave-One-Out cross-validation
y_true = []
y_pred = []

for train_index, test_index in loo.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    best_model = ElasticNet(alpha=best_params['alpha'], l1_ratio=best_params['l1_ratio'], max_iter=10000, random_state=seed)
    best_model.fit(X_train_scaled, y_train)
    
    y_test_pred = best_model.predict(X_test_scaled)
    
    y_true.append(y_test[0])
    y_pred.append(y_test_pred[0])

# Convert lists to numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Calculate the Pearson correlation coefficient
pearson_corr, _ = pearsonr(y_true, y_pred)

# Calculate the R-squared value
r_squared = r2_score(y_true, y_pred)

# Calculate the mean absolute error
mae = mean_absolute_error(y_true, y_pred)

print(f"Mean MAE: {mae:.3f}")
print(f"Pearson correlation: {pearson_corr:.3f}")
print(f"R-squared: {r_squared:.3f}")

In [114]:
loo = LeaveOneOut()

# Initialize lists to store actual and predicted values
y_true = []
y_pred = []

# Loop over each train-test split
for train_index, test_index in loo.split(X):
    # Split the data
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Scale the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Define and train the model
    model = ElasticNet(alpha=0.01, l1_ratio=1.0, max_iter=100000, random_state=seed)
    model.fit(X_train_scaled, y_train)
    #model.fit(X_train, y_train)
    
    # Make prediction on the test set
    y_test_pred = model.predict(X_test_scaled)
    #y_test_pred = model.predict(X_test)
    
    # Store the actual and predicted values
    y_true.append(y_test[0])
    y_pred.append(y_test_pred[0])

# Convert lists to numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Calculate the Pearson correlation coefficient
pearson_corr, _ = pearsonr(y_true, y_pred)

# Calculate the R-squared value
r_squared = r2_score(y_true, y_pred)

# Calculate the mean absolute error
mae = mean_absolute_error(y_true, y_pred)

print(f"Mean MAE: {mae:.3f}")
print(f"Pearson correlation: {pearson_corr:.3f}")
print(f"R-squared: {r_squared:.3f}")

Mean MAE: 2.265
Pearson correlation: 0.939
R-squared: 0.872


# Bone Marrow

In [115]:
bone_df_metadata = df_new_metadata[df_new_metadata["source.name"].str.startswith("Bone")]

bone_df_metadata.head(3)

,Unnamed: 0,Sample.name,title,source.name,organism,characteristics..age,characteristics..developmental.stage,characteristics..sex,molecule,description,processed.data.file,raw.file,BioSample,Instrument.Model
Sample.name.new,,,,,,,,,,,,,,
Bone_15-m-15,A14_384Bulk_Plate1_S14,A14_384Bulk_Plate1_S14,Tabula Muris Senis (bulk RNA seq),Bone_15,Mus musculus C57/BL6,15,months postnatal,m,total RNA,NaN,190214_A00111_0269_AHH3J3DSXX__190214_A00111_0...,SRR9127024,SAMN11854580,Illumina NovaSeq 6000
Bone_45-m-15,A8_384Bulk_Plate1_S8,A8_384Bulk_Plate1_S8,Tabula Muris Senis (bulk RNA seq),Bone_45,Mus musculus C57/BL6,15,months postnatal,m,total RNA,NaN,190214_A00111_0269_AHH3J3DSXX__190214_A00111_0...,SRR9127051,SAMN11854619,Illumina NovaSeq 6000
Bone_36-f-6,B13_384Bulk_Plate1_S37,B13_384Bulk_Plate1_S37,Tabula Muris Senis (bulk RNA seq),Bone_36,Mus musculus C57/BL6,6,months postnatal,f,total RNA,NaN,190214_A00111_0269_AHH3J3DSXX__190214_A00111_0...,SRR9126850,SAMN11854637,Illumina NovaSeq 6000


In [116]:
bone_df_counts = df.loc[bone_df_metadata.index.tolist(),]

bone_df_counts.head(3)

Gene,ENSMUSG00000109644,ENSMUSG00000007777,ENSMUSG00000043644,ENSMUSG00000042208,ENSMUSG00000020831,ENSMUSG00000107002,ENSMUSG00000058706,ENSMUSG00000099146,ENSMUSG00000028608,ENSMUSG00000097882,...,ENSMUSG00000096015,ENSMUSG00000096037,ENSMUSG00000095183,ENSMUSG00000094760,ENSMUSG00000095749,ENSMUSG00000094657,ENSMUSG00000095645,ENSMUSG00000096765,ENSMUSG00000096170,ENSMUSG00000095453
Bone_15-m-15,26,39,5,96,31,49,26,25,123,8,...,0,0,0,0,0,0,0,0,0,0
Bone_45-m-15,35,27,22,163,19,46,8,26,172,27,...,0,0,0,0,0,0,0,0,0,0
Bone_36-f-6,16,63,9,141,30,141,24,9,158,20,...,0,0,0,0,0,0,0,0,0,0


In [117]:
bone_df_age = bone_df_counts.copy()

In [118]:
bone_df_age["age"] = bone_df_metadata["characteristics..age"].tolist()

In [119]:
bone_df_age.head(5)

Gene,ENSMUSG00000109644,ENSMUSG00000007777,ENSMUSG00000043644,ENSMUSG00000042208,ENSMUSG00000020831,ENSMUSG00000107002,ENSMUSG00000058706,ENSMUSG00000099146,ENSMUSG00000028608,ENSMUSG00000097882,...,ENSMUSG00000096037,ENSMUSG00000095183,ENSMUSG00000094760,ENSMUSG00000095749,ENSMUSG00000094657,ENSMUSG00000095645,ENSMUSG00000096765,ENSMUSG00000096170,ENSMUSG00000095453,age
Bone_15-m-15,26,39,5,96,31,49,26,25,123,8,...,0,0,0,0,0,0,0,0,0,15
Bone_45-m-15,35,27,22,163,19,46,8,26,172,27,...,0,0,0,0,0,0,0,0,0,15
Bone_36-f-6,16,63,9,141,30,141,24,9,158,20,...,0,0,0,0,0,0,0,0,0,6
Bone_4-f-15,4,36,7,81,17,82,13,12,100,9,...,0,0,0,0,0,0,0,0,0,15
Bone_10-m-12,8,85,19,203,40,141,142,7,259,12,...,0,0,0,0,0,0,0,0,0,12


In [120]:
bone_df_age["age"] = pd.to_numeric(bone_df_metadata["characteristics..age"].tolist(), errors="coerce")

In [121]:
bone_df_age.head(3)

Gene,ENSMUSG00000109644,ENSMUSG00000007777,ENSMUSG00000043644,ENSMUSG00000042208,ENSMUSG00000020831,ENSMUSG00000107002,ENSMUSG00000058706,ENSMUSG00000099146,ENSMUSG00000028608,ENSMUSG00000097882,...,ENSMUSG00000096037,ENSMUSG00000095183,ENSMUSG00000094760,ENSMUSG00000095749,ENSMUSG00000094657,ENSMUSG00000095645,ENSMUSG00000096765,ENSMUSG00000096170,ENSMUSG00000095453,age
Bone_15-m-15,26,39,5,96,31,49,26,25,123,8,...,0,0,0,0,0,0,0,0,0,15
Bone_45-m-15,35,27,22,163,19,46,8,26,172,27,...,0,0,0,0,0,0,0,0,0,15
Bone_36-f-6,16,63,9,141,30,141,24,9,158,20,...,0,0,0,0,0,0,0,0,0,6


In [122]:
bone_df_age["age"] = pd.to_numeric(bone_df_metadata["characteristics..age"].tolist(), errors="coerce")

In [123]:
bone_df_age.head(3)

Gene,ENSMUSG00000109644,ENSMUSG00000007777,ENSMUSG00000043644,ENSMUSG00000042208,ENSMUSG00000020831,ENSMUSG00000107002,ENSMUSG00000058706,ENSMUSG00000099146,ENSMUSG00000028608,ENSMUSG00000097882,...,ENSMUSG00000096037,ENSMUSG00000095183,ENSMUSG00000094760,ENSMUSG00000095749,ENSMUSG00000094657,ENSMUSG00000095645,ENSMUSG00000096765,ENSMUSG00000096170,ENSMUSG00000095453,age
Bone_15-m-15,26,39,5,96,31,49,26,25,123,8,...,0,0,0,0,0,0,0,0,0,15
Bone_45-m-15,35,27,22,163,19,46,8,26,172,27,...,0,0,0,0,0,0,0,0,0,15
Bone_36-f-6,16,63,9,141,30,141,24,9,158,20,...,0,0,0,0,0,0,0,0,0,6


In [124]:
bone_log_transformed_data = np.log1p(bone_df_counts)

In [125]:
bone_data = bone_df_age.values

In [126]:
X = bone_log_transformed_data.values
y = bone_data[:,-1]

In [ ]:
## Define the parameter grid
param_grid = {
    'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1.0, 10.0, 100.0],
    'l1_ratio': np.arange(0, 1.1, 0.1)
}

# Define the model
model = ElasticNet(max_iter=10000, random_state=seed)

# Define Leave-One-Out cross-validation
loo = LeaveOneOut()

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=loo, scoring='neg_mean_absolute_error', n_jobs=-1)

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Fit the GridSearchCV
grid_search.fit(X_scaled, y)

# Get the best parameters
best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

# Train the model with the best parameters using Leave-One-Out cross-validation
y_true = []
y_pred = []

for train_index, test_index in loo.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    best_model = ElasticNet(alpha=best_params['alpha'], l1_ratio=best_params['l1_ratio'], max_iter=10000, random_state=seed)
    best_model.fit(X_train_scaled, y_train)
    
    y_test_pred = best_model.predict(X_test_scaled)
    
    y_true.append(y_test[0])
    y_pred.append(y_test_pred[0])

# Convert lists to numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Calculate the Pearson correlation coefficient
pearson_corr, _ = pearsonr(y_true, y_pred)

# Calculate the R-squared value
r_squared = r2_score(y_true, y_pred)

# Calculate the mean absolute error
mae = mean_absolute_error(y_true, y_pred)

print(f"Mean MAE: {mae:.3f}")
print(f"Pearson correlation: {pearson_corr:.3f}")
print(f"R-squared: {r_squared:.3f}")

In [127]:
loo = LeaveOneOut()

# Initialize lists to store actual and predicted values
y_true = []
y_pred = []

# Loop over each train-test split
for train_index, test_index in loo.split(X):
    # Split the data
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Scale the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Define and train the model
    model = ElasticNet(alpha=0.01, l1_ratio=0.30000000000000004, max_iter=100000, random_state=seed)
    model.fit(X_train_scaled, y_train)
    #model.fit(X_train, y_train)
    
    # Make prediction on the test set
    y_test_pred = model.predict(X_test_scaled)
    #y_test_pred = model.predict(X_test)
    
    # Store the actual and predicted values
    y_true.append(y_test[0])
    y_pred.append(y_test_pred[0])

# Convert lists to numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Calculate the Pearson correlation coefficient
pearson_corr, _ = pearsonr(y_true, y_pred)

# Calculate the R-squared value
r_squared = r2_score(y_true, y_pred)

# Calculate the mean absolute error
mae = mean_absolute_error(y_true, y_pred)

print(f"Mean MAE: {mae:.3f}")
print(f"Pearson correlation: {pearson_corr:.3f}")
print(f"R-squared: {r_squared:.3f}")

Mean MAE: 3.906
Pearson correlation: 0.833
R-squared: 0.633


# MAT

In [128]:
mat_df_metadata = df_new_metadata[df_new_metadata["source.name"].str.startswith("MAT")]

mat_df_metadata.head(3)

,Unnamed: 0,Sample.name,title,source.name,organism,characteristics..age,characteristics..developmental.stage,characteristics..sex,molecule,description,processed.data.file,raw.file,BioSample,Instrument.Model
Sample.name.new,,,,,,,,,,,,,,
MAT_15-m-18,A15_384Bulk_Plate2_S15,A15_384Bulk_Plate2_S15,Tabula Muris Senis (bulk RNA seq),MAT_15,Mus musculus C57/BL6,18,months postnatal,m,total RNA,NaN,190214_A00111_0269_AHH3J3DSXX__190214_A00111_0...,SRR9127020,SAMN11854584,Illumina NovaSeq 6000
MAT_4-m-1,A21_384Bulk_Plate1_S21,A21_384Bulk_Plate1_S21,Tabula Muris Senis (bulk RNA seq),MAT_4,Mus musculus C57/BL6,1,months postnatal,m,total RNA,NaN,190214_A00111_0269_AHH3J3DSXX__190214_A00111_0...,SRR9127502,SAMN11854598,Illumina NovaSeq 6000
MAT_42-f-1,A23_384Bulk_Plate2_S23,A23_384Bulk_Plate2_S23,Tabula Muris Senis (bulk RNA seq),MAT_42,Mus musculus C57/BL6,1,months postnatal,f,total RNA,NaN,190214_A00111_0269_AHH3J3DSXX__190214_A00111_0...,SRR9127507,SAMN11854603,Illumina NovaSeq 6000


In [129]:
mat_df_counts = df.loc[mat_df_metadata.index.tolist(),]

mat_df_counts.head(3)

Gene,ENSMUSG00000109644,ENSMUSG00000007777,ENSMUSG00000043644,ENSMUSG00000042208,ENSMUSG00000020831,ENSMUSG00000107002,ENSMUSG00000058706,ENSMUSG00000099146,ENSMUSG00000028608,ENSMUSG00000097882,...,ENSMUSG00000096015,ENSMUSG00000096037,ENSMUSG00000095183,ENSMUSG00000094760,ENSMUSG00000095749,ENSMUSG00000094657,ENSMUSG00000095645,ENSMUSG00000096765,ENSMUSG00000096170,ENSMUSG00000095453
MAT_15-m-18,7,195,37,294,36,245,52,7,413,19,...,0,0,0,0,0,0,0,0,0,0
MAT_4-m-1,5,222,77,293,53,377,60,3,615,18,...,0,0,0,0,0,0,0,0,0,0
MAT_42-f-1,1,253,92,462,53,399,45,6,573,38,...,0,0,0,0,0,0,0,0,0,0


In [130]:
mat_df_age = mat_df_counts.copy()

In [131]:
mat_df_age["age"] = mat_df_metadata["characteristics..age"].tolist()

In [132]:
mat_df_age.head(5)

Gene,ENSMUSG00000109644,ENSMUSG00000007777,ENSMUSG00000043644,ENSMUSG00000042208,ENSMUSG00000020831,ENSMUSG00000107002,ENSMUSG00000058706,ENSMUSG00000099146,ENSMUSG00000028608,ENSMUSG00000097882,...,ENSMUSG00000096037,ENSMUSG00000095183,ENSMUSG00000094760,ENSMUSG00000095749,ENSMUSG00000094657,ENSMUSG00000095645,ENSMUSG00000096765,ENSMUSG00000096170,ENSMUSG00000095453,age
MAT_15-m-18,7,195,37,294,36,245,52,7,413,19,...,0,0,0,0,0,0,0,0,0,18
MAT_4-m-1,5,222,77,293,53,377,60,3,615,18,...,0,0,0,0,0,0,0,0,0,1
MAT_42-f-1,1,253,92,462,53,399,45,6,573,38,...,0,0,0,0,0,0,0,0,0,1
MAT_52-m-15,11,141,52,449,56,189,55,10,461,20,...,0,0,0,0,0,0,0,0,0,15
MAT_2-f-9,38,141,37,271,73,268,48,10,446,23,...,0,0,0,0,0,0,0,0,0,9


In [133]:
mat_df_age["age"] = pd.to_numeric(mat_df_metadata["characteristics..age"].tolist(), errors="coerce")

In [134]:
mat_df_age.head(3)

Gene,ENSMUSG00000109644,ENSMUSG00000007777,ENSMUSG00000043644,ENSMUSG00000042208,ENSMUSG00000020831,ENSMUSG00000107002,ENSMUSG00000058706,ENSMUSG00000099146,ENSMUSG00000028608,ENSMUSG00000097882,...,ENSMUSG00000096037,ENSMUSG00000095183,ENSMUSG00000094760,ENSMUSG00000095749,ENSMUSG00000094657,ENSMUSG00000095645,ENSMUSG00000096765,ENSMUSG00000096170,ENSMUSG00000095453,age
MAT_15-m-18,7,195,37,294,36,245,52,7,413,19,...,0,0,0,0,0,0,0,0,0,18
MAT_4-m-1,5,222,77,293,53,377,60,3,615,18,...,0,0,0,0,0,0,0,0,0,1
MAT_42-f-1,1,253,92,462,53,399,45,6,573,38,...,0,0,0,0,0,0,0,0,0,1


In [135]:
mat_log_transformed_data = np.log1p(mat_df_counts)

In [136]:
mat_data = mat_df_age.values

In [137]:
X = mat_log_transformed_data.values
y = mat_data[:,-1]

In [ ]:
## Define the parameter grid
param_grid = {
    'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1.0, 10.0, 100.0],
    'l1_ratio': np.arange(0, 1.1, 0.1)
}

# Define the model
model = ElasticNet(max_iter=10000, random_state=seed)

# Define Leave-One-Out cross-validation
loo = LeaveOneOut()

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=loo, scoring='neg_mean_absolute_error', n_jobs=-1)

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Fit the GridSearchCV
grid_search.fit(X_scaled, y)

# Get the best parameters
best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

# Train the model with the best parameters using Leave-One-Out cross-validation
y_true = []
y_pred = []

for train_index, test_index in loo.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    best_model = ElasticNet(alpha=best_params['alpha'], l1_ratio=best_params['l1_ratio'], max_iter=10000, random_state=seed)
    best_model.fit(X_train_scaled, y_train)
    
    y_test_pred = best_model.predict(X_test_scaled)
    
    y_true.append(y_test[0])
    y_pred.append(y_test_pred[0])

# Convert lists to numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Calculate the Pearson correlation coefficient
pearson_corr, _ = pearsonr(y_true, y_pred)

# Calculate the R-squared value
r_squared = r2_score(y_true, y_pred)

# Calculate the mean absolute error
mae = mean_absolute_error(y_true, y_pred)

print(f"Mean MAE: {mae:.3f}")
print(f"Pearson correlation: {pearson_corr:.3f}")
print(f"R-squared: {r_squared:.3f}")

In [138]:
loo = LeaveOneOut()

# Initialize lists to store actual and predicted values
y_true = []
y_pred = []

# Loop over each train-test split
for train_index, test_index in loo.split(X):
    # Split the data
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Scale the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Define and train the model
    model = ElasticNet(alpha=1.0, l1_ratio=1.0, max_iter=100000, random_state=seed)
    model.fit(X_train_scaled, y_train)
    #model.fit(X_train, y_train)
    
    # Make prediction on the test set
    y_test_pred = model.predict(X_test_scaled)
    #y_test_pred = model.predict(X_test)
    
    # Store the actual and predicted values
    y_true.append(y_test[0])
    y_pred.append(y_test_pred[0])

# Convert lists to numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Calculate the Pearson correlation coefficient
pearson_corr, _ = pearsonr(y_true, y_pred)

# Calculate the R-squared value
r_squared = r2_score(y_true, y_pred)

# Calculate the mean absolute error
mae = mean_absolute_error(y_true, y_pred)

print(f"Mean MAE: {mae:.3f}")
print(f"Pearson correlation: {pearson_corr:.3f}")
print(f"R-squared: {r_squared:.3f}")

Mean MAE: 2.964
Pearson correlation: 0.922
R-squared: 0.814


# Pancreas

In [139]:
pancreas_df_metadata = df_new_metadata[df_new_metadata["source.name"].str.startswith("Pancreas")]

pancreas_df_metadata.head(3)

,Unnamed: 0,Sample.name,title,source.name,organism,characteristics..age,characteristics..developmental.stage,characteristics..sex,molecule,description,processed.data.file,raw.file,BioSample,Instrument.Model
Sample.name.new,,,,,,,,,,,,,,
Pancreas_20-m-15,A13_384Bulk_Plate2_S13,A13_384Bulk_Plate2_S13,Tabula Muris Senis (bulk RNA seq),Pancreas_20,Mus musculus C57/BL6,15,months postnatal,m,total RNA,NaN,190214_A00111_0269_AHH3J3DSXX__190214_A00111_0...,SRR9127026,SAMN11854578,Illumina NovaSeq 6000
Pancreas_49-f-12,A16_384Bulk_Plate2_S16,A16_384Bulk_Plate2_S16,Tabula Muris Senis (bulk RNA seq),Pancreas_49,Mus musculus C57/BL6,12,months postnatal,f,total RNA,NaN,190214_A00111_0269_AHH3J3DSXX__190214_A00111_0...,SRR9127268,SAMN11854586,Illumina NovaSeq 6000
Pancreas_19-m-27,A6_384Bulk_Plate2_S6,A6_384Bulk_Plate2_S6,Tabula Muris Senis (bulk RNA seq),Pancreas_19,Mus musculus C57/BL6,27,months postnatal,m,total RNA,NaN,190214_A00111_0269_AHH3J3DSXX__190214_A00111_0...,SRR9127259,SAMN11854614,Illumina NovaSeq 6000


In [140]:
pancreas_df_counts = df.loc[pancreas_df_metadata.index.tolist(),]

pancreas_df_counts.head(3)

Gene,ENSMUSG00000109644,ENSMUSG00000007777,ENSMUSG00000043644,ENSMUSG00000042208,ENSMUSG00000020831,ENSMUSG00000107002,ENSMUSG00000058706,ENSMUSG00000099146,ENSMUSG00000028608,ENSMUSG00000097882,...,ENSMUSG00000096015,ENSMUSG00000096037,ENSMUSG00000095183,ENSMUSG00000094760,ENSMUSG00000095749,ENSMUSG00000094657,ENSMUSG00000095645,ENSMUSG00000096765,ENSMUSG00000096170,ENSMUSG00000095453
Pancreas_20-m-15,2,10,5,57,2,30,1,1,43,0,...,0,0,0,0,0,0,0,0,0,0
Pancreas_49-f-12,0,10,0,19,10,26,3,0,42,0,...,0,0,0,0,0,0,0,0,0,0
Pancreas_19-m-27,2,23,5,55,11,69,3,0,62,3,...,0,0,0,0,0,0,0,0,0,0


In [141]:
pancreas_df_age = pancreas_df_counts.copy()

In [142]:
pancreas_df_age["age"] = pancreas_df_metadata["characteristics..age"].tolist()

In [143]:
pancreas_df_age.head(5)

Gene,ENSMUSG00000109644,ENSMUSG00000007777,ENSMUSG00000043644,ENSMUSG00000042208,ENSMUSG00000020831,ENSMUSG00000107002,ENSMUSG00000058706,ENSMUSG00000099146,ENSMUSG00000028608,ENSMUSG00000097882,...,ENSMUSG00000096037,ENSMUSG00000095183,ENSMUSG00000094760,ENSMUSG00000095749,ENSMUSG00000094657,ENSMUSG00000095645,ENSMUSG00000096765,ENSMUSG00000096170,ENSMUSG00000095453,age
Pancreas_20-m-15,2,10,5,57,2,30,1,1,43,0,...,0,0,0,0,0,0,0,0,0,15
Pancreas_49-f-12,0,10,0,19,10,26,3,0,42,0,...,0,0,0,0,0,0,0,0,0,12
Pancreas_19-m-27,2,23,5,55,11,69,3,0,62,3,...,0,0,0,0,0,0,0,0,0,27
Pancreas_37-f-12,2,42,0,57,12,71,6,3,96,5,...,0,0,0,0,0,0,0,0,0,12
Pancreas_46-f-3,1,12,0,19,4,1,13,2,28,1,...,0,0,0,0,0,0,0,0,0,3


In [144]:
pancreas_df_age["age"] = pd.to_numeric(pancreas_df_metadata["characteristics..age"].tolist(), errors="coerce")

In [145]:
pancreas_df_age.head(3)

Gene,ENSMUSG00000109644,ENSMUSG00000007777,ENSMUSG00000043644,ENSMUSG00000042208,ENSMUSG00000020831,ENSMUSG00000107002,ENSMUSG00000058706,ENSMUSG00000099146,ENSMUSG00000028608,ENSMUSG00000097882,...,ENSMUSG00000096037,ENSMUSG00000095183,ENSMUSG00000094760,ENSMUSG00000095749,ENSMUSG00000094657,ENSMUSG00000095645,ENSMUSG00000096765,ENSMUSG00000096170,ENSMUSG00000095453,age
Pancreas_20-m-15,2,10,5,57,2,30,1,1,43,0,...,0,0,0,0,0,0,0,0,0,15
Pancreas_49-f-12,0,10,0,19,10,26,3,0,42,0,...,0,0,0,0,0,0,0,0,0,12
Pancreas_19-m-27,2,23,5,55,11,69,3,0,62,3,...,0,0,0,0,0,0,0,0,0,27


In [146]:
pancreas_log_transformed_data = np.log1p(pancreas_df_counts)

In [147]:
pancreas_data = pancreas_df_age.values

In [148]:
X = pancreas_log_transformed_data.values
y = pancreas_data[:,-1]

In [ ]:
## Define the parameter grid
param_grid = {
    'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1.0, 10.0, 100.0],
    'l1_ratio': np.arange(0, 1.1, 0.1)
}

# Define the model
model = ElasticNet(max_iter=10000, random_state=seed)

# Define Leave-One-Out cross-validation
loo = LeaveOneOut()

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=loo, scoring='neg_mean_absolute_error', n_jobs=-1)

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Fit the GridSearchCV
grid_search.fit(X_scaled, y)

# Get the best parameters
best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

# Train the model with the best parameters using Leave-One-Out cross-validation
y_true = []
y_pred = []

for train_index, test_index in loo.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    best_model = ElasticNet(alpha=best_params['alpha'], l1_ratio=best_params['l1_ratio'], max_iter=10000, random_state=seed)
    best_model.fit(X_train_scaled, y_train)
    
    y_test_pred = best_model.predict(X_test_scaled)
    
    y_true.append(y_test[0])
    y_pred.append(y_test_pred[0])

# Convert lists to numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Calculate the Pearson correlation coefficient
pearson_corr, _ = pearsonr(y_true, y_pred)

# Calculate the R-squared value
r_squared = r2_score(y_true, y_pred)

# Calculate the mean absolute error
mae = mean_absolute_error(y_true, y_pred)

print(f"Mean MAE: {mae:.3f}")
print(f"Pearson correlation: {pearson_corr:.3f}")
print(f"R-squared: {r_squared:.3f}")

In [149]:
loo = LeaveOneOut()

# Initialize lists to store actual and predicted values
y_true = []
y_pred = []

# Loop over each train-test split
for train_index, test_index in loo.split(X):
    # Split the data
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Scale the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Define and train the model
    model = ElasticNet(alpha=0.1, l1_ratio=1.0, max_iter=100000, random_state=seed)
    model.fit(X_train_scaled, y_train)
    #model.fit(X_train, y_train)
    
    # Make prediction on the test set
    y_test_pred = model.predict(X_test_scaled)
    #y_test_pred = model.predict(X_test)
    
    # Store the actual and predicted values
    y_true.append(y_test[0])
    y_pred.append(y_test_pred[0])

# Convert lists to numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Calculate the Pearson correlation coefficient
pearson_corr, _ = pearsonr(y_true, y_pred)

# Calculate the R-squared value
r_squared = r2_score(y_true, y_pred)

# Calculate the mean absolute error
mae = mean_absolute_error(y_true, y_pred)

print(f"Mean MAE: {mae:.3f}")
print(f"Pearson correlation: {pearson_corr:.3f}")
print(f"R-squared: {r_squared:.3f}")

Mean MAE: 3.398
Pearson correlation: 0.893
R-squared: 0.734


# SCAT

In [150]:
scat_df_metadata = df_new_metadata[df_new_metadata["source.name"].str.startswith("SCAT")]

scat_df_metadata.head(3)

,Unnamed: 0,Sample.name,title,source.name,organism,characteristics..age,characteristics..developmental.stage,characteristics..sex,molecule,description,processed.data.file,raw.file,BioSample,Instrument.Model
Sample.name.new,,,,,,,,,,,,,,
SCAT_2-f-9,A14_384Bulk_Plate2_S14,A14_384Bulk_Plate2_S14,Tabula Muris Senis (bulk RNA seq),SCAT_2,Mus musculus C57/BL6,9,months postnatal,f,total RNA,NaN,190214_A00111_0269_AHH3J3DSXX__190214_A00111_0...,SRR9127025,SAMN11854581,Illumina NovaSeq 6000
SCAT_43-m-3,A1_384Bulk_Plate3_S1,A1_384Bulk_Plate3_S1,Tabula Muris Senis (bulk RNA seq),SCAT_43,Mus musculus C57/BL6,3,months postnatal,m,total RNA,NaN,190214_A00111_0269_AHH3J3DSXX__190214_A00111_0...,SRR9127206,SAMN11854567,Illumina NovaSeq 6000
SCAT_37-f-6,B11_384Bulk_Plate3_S35,B11_384Bulk_Plate3_S35,Tabula Muris Senis (bulk RNA seq),SCAT_37,Mus musculus C57/BL6,6,months postnatal,f,total RNA,NaN,190214_A00111_0269_AHH3J3DSXX__190214_A00111_0...,SRR9126800,SAMN11854633,Illumina NovaSeq 6000


In [151]:
scat_df_counts = df.loc[scat_df_metadata.index.tolist(),]

scat_df_counts.head(3)

Gene,ENSMUSG00000109644,ENSMUSG00000007777,ENSMUSG00000043644,ENSMUSG00000042208,ENSMUSG00000020831,ENSMUSG00000107002,ENSMUSG00000058706,ENSMUSG00000099146,ENSMUSG00000028608,ENSMUSG00000097882,...,ENSMUSG00000096015,ENSMUSG00000096037,ENSMUSG00000095183,ENSMUSG00000094760,ENSMUSG00000095749,ENSMUSG00000094657,ENSMUSG00000095645,ENSMUSG00000096765,ENSMUSG00000096170,ENSMUSG00000095453
SCAT_2-f-9,13,146,33,272,63,269,65,15,497,27,...,0,0,0,0,0,0,0,0,0,0
SCAT_43-m-3,29,238,37,570,123,593,97,14,801,39,...,0,0,0,0,0,0,0,0,0,0
SCAT_37-f-6,7,180,36,354,45,346,59,6,475,14,...,0,0,0,0,0,0,0,0,0,0


In [152]:
scat_df_age = scat_df_counts.copy()

In [153]:
scat_df_age["age"] = scat_df_metadata["characteristics..age"].tolist()

In [154]:
scat_df_age.head(5)

Gene,ENSMUSG00000109644,ENSMUSG00000007777,ENSMUSG00000043644,ENSMUSG00000042208,ENSMUSG00000020831,ENSMUSG00000107002,ENSMUSG00000058706,ENSMUSG00000099146,ENSMUSG00000028608,ENSMUSG00000097882,...,ENSMUSG00000096037,ENSMUSG00000095183,ENSMUSG00000094760,ENSMUSG00000095749,ENSMUSG00000094657,ENSMUSG00000095645,ENSMUSG00000096765,ENSMUSG00000096170,ENSMUSG00000095453,age
SCAT_2-f-9,13,146,33,272,63,269,65,15,497,27,...,0,0,0,0,0,0,0,0,0,9
SCAT_43-m-3,29,238,37,570,123,593,97,14,801,39,...,0,0,0,0,0,0,0,0,0,3
SCAT_37-f-6,7,180,36,354,45,346,59,6,475,14,...,0,0,0,0,0,0,0,0,0,6
SCAT_12-f-3,26,246,29,442,91,284,71,9,641,15,...,0,0,0,0,0,0,0,0,0,3
SCAT_18-m-24,30,89,45,274,26,259,53,34,253,11,...,0,0,0,0,0,0,0,0,0,24


In [155]:
scat_df_age["age"] = pd.to_numeric(scat_df_metadata["characteristics..age"].tolist(), errors="coerce")

In [156]:
scat_df_age.head(3)

Gene,ENSMUSG00000109644,ENSMUSG00000007777,ENSMUSG00000043644,ENSMUSG00000042208,ENSMUSG00000020831,ENSMUSG00000107002,ENSMUSG00000058706,ENSMUSG00000099146,ENSMUSG00000028608,ENSMUSG00000097882,...,ENSMUSG00000096037,ENSMUSG00000095183,ENSMUSG00000094760,ENSMUSG00000095749,ENSMUSG00000094657,ENSMUSG00000095645,ENSMUSG00000096765,ENSMUSG00000096170,ENSMUSG00000095453,age
SCAT_2-f-9,13,146,33,272,63,269,65,15,497,27,...,0,0,0,0,0,0,0,0,0,9
SCAT_43-m-3,29,238,37,570,123,593,97,14,801,39,...,0,0,0,0,0,0,0,0,0,3
SCAT_37-f-6,7,180,36,354,45,346,59,6,475,14,...,0,0,0,0,0,0,0,0,0,6


In [157]:
scat_log_transformed_data = np.log1p(scat_df_counts)

In [158]:
scat_data = scat_df_age.values

In [159]:
X = scat_log_transformed_data.values
y = scat_data[:,-1]

In [ ]:
## Define the parameter grid
param_grid = {
    'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1.0, 10.0, 100.0],
    'l1_ratio': np.arange(0, 1.1, 0.1)
}

# Define the model
model = ElasticNet(max_iter=10000)

# Define Leave-One-Out cross-validation
loo = LeaveOneOut()

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=loo, scoring='neg_mean_absolute_error', n_jobs=-1)

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Fit the GridSearchCV
grid_search.fit(X_scaled, y)

# Get the best parameters
best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

# Train the model with the best parameters using Leave-One-Out cross-validation
y_true = []
y_pred = []

for train_index, test_index in loo.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    best_model = ElasticNet(alpha=best_params['alpha'], l1_ratio=best_params['l1_ratio'], max_iter=10000)
    best_model.fit(X_train_scaled, y_train)
    
    y_test_pred = best_model.predict(X_test_scaled)
    
    y_true.append(y_test[0])
    y_pred.append(y_test_pred[0])

# Convert lists to numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Calculate the Pearson correlation coefficient
pearson_corr, _ = pearsonr(y_true, y_pred)

# Calculate the R-squared value
r_squared = r2_score(y_true, y_pred)

# Calculate the mean absolute error
mae = mean_absolute_error(y_true, y_pred)

print(f"Mean MAE: {mae:.3f}")
print(f"Pearson correlation: {pearson_corr:.3f}")
print(f"R-squared: {r_squared:.3f}")

In [160]:
loo = LeaveOneOut()

# Initialize lists to store actual and predicted values
y_true = []
y_pred = []

# Loop over each train-test split
for train_index, test_index in loo.split(X):
    # Split the data
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Scale the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Define and train the model
    model = ElasticNet(alpha=0.001, l1_ratio=0.9, max_iter=100000, random_state=seed)
    model.fit(X_train_scaled, y_train)
    #model.fit(X_train, y_train)
    
    # Make prediction on the test set
    y_test_pred = model.predict(X_test_scaled)
    #y_test_pred = model.predict(X_test)
    
    # Store the actual and predicted values
    y_true.append(y_test[0])
    y_pred.append(y_test_pred[0])

# Convert lists to numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Calculate the Pearson correlation coefficient
pearson_corr, _ = pearsonr(y_true, y_pred)

# Calculate the R-squared value
r_squared = r2_score(y_true, y_pred)

# Calculate the mean absolute error
mae = mean_absolute_error(y_true, y_pred)

print(f"Mean MAE: {mae:.3f}")
print(f"Pearson correlation: {pearson_corr:.3f}")
print(f"R-squared: {r_squared:.3f}")

Mean MAE: 3.069
Pearson correlation: 0.879
R-squared: 0.768


# Skin

In [161]:
skin_df_metadata = df_new_metadata[df_new_metadata["source.name"].str.startswith("Skin")]

skin_df_metadata.head(3)

,Unnamed: 0,Sample.name,title,source.name,organism,characteristics..age,characteristics..developmental.stage,characteristics..sex,molecule,description,processed.data.file,raw.file,BioSample,Instrument.Model
Sample.name.new,,,,,,,,,,,,,,
Skin_39-m-9,A3_384Bulk_Plate3_S3,A3_384Bulk_Plate3_S3,Tabula Muris Senis (bulk RNA seq),Skin_39,Mus musculus C57/BL6,9,months postnatal,m,total RNA,NaN,190214_A00111_0269_AHH3J3DSXX__190214_A00111_0...,SRR9127251,SAMN11854606,Illumina NovaSeq 6000
Skin_23-m-1,B11_384Bulk_Plate2_S35,B11_384Bulk_Plate2_S35,Tabula Muris Senis (bulk RNA seq),Skin_23,Mus musculus C57/BL6,1,months postnatal,m,total RNA,NaN,190214_A00111_0269_AHH3J3DSXX__190214_A00111_0...,SRR9126801,SAMN11854632,Illumina NovaSeq 6000
Skin_9-m-27,B4_384Bulk_Plate2_S28,B4_384Bulk_Plate2_S28,Tabula Muris Senis (bulk RNA seq),Skin_9,Mus musculus C57/BL6,27,months postnatal,m,total RNA,NaN,190214_A00111_0269_AHH3J3DSXX__190214_A00111_0...,SRR9127390,SAMN11854668,Illumina NovaSeq 6000


In [162]:
skin_df_counts = df.loc[skin_df_metadata.index.tolist(),]

skin_df_counts.head(3)

Gene,ENSMUSG00000109644,ENSMUSG00000007777,ENSMUSG00000043644,ENSMUSG00000042208,ENSMUSG00000020831,ENSMUSG00000107002,ENSMUSG00000058706,ENSMUSG00000099146,ENSMUSG00000028608,ENSMUSG00000097882,...,ENSMUSG00000096015,ENSMUSG00000096037,ENSMUSG00000095183,ENSMUSG00000094760,ENSMUSG00000095749,ENSMUSG00000094657,ENSMUSG00000095645,ENSMUSG00000096765,ENSMUSG00000096170,ENSMUSG00000095453
Skin_39-m-9,27,78,37,138,34,222,8,24,300,23,...,0,0,0,0,0,0,0,0,0,0
Skin_23-m-1,10,56,26,167,35,216,38,17,247,29,...,0,0,0,0,0,0,0,0,0,0
Skin_9-m-27,0,75,12,130,45,117,47,22,234,45,...,0,0,0,0,0,0,0,0,0,0


In [163]:
skin_df_age = skin_df_counts.copy()

In [164]:
skin_df_age["age"] = skin_df_metadata["characteristics..age"].tolist()

In [165]:
skin_df_age.head(5)

Gene,ENSMUSG00000109644,ENSMUSG00000007777,ENSMUSG00000043644,ENSMUSG00000042208,ENSMUSG00000020831,ENSMUSG00000107002,ENSMUSG00000058706,ENSMUSG00000099146,ENSMUSG00000028608,ENSMUSG00000097882,...,ENSMUSG00000096037,ENSMUSG00000095183,ENSMUSG00000094760,ENSMUSG00000095749,ENSMUSG00000094657,ENSMUSG00000095645,ENSMUSG00000096765,ENSMUSG00000096170,ENSMUSG00000095453,age
Skin_39-m-9,27,78,37,138,34,222,8,24,300,23,...,0,0,0,0,0,0,0,0,0,9
Skin_23-m-1,10,56,26,167,35,216,38,17,247,29,...,0,0,0,0,0,0,0,0,0,1
Skin_9-m-27,0,75,12,130,45,117,47,22,234,45,...,0,0,0,0,0,0,0,0,0,27
Skin_22-m-3,1,30,0,71,20,64,10,18,115,13,...,0,0,0,0,0,0,0,0,0,3
Skin_38-m-6,15,34,6,73,40,98,12,17,136,7,...,0,0,0,0,0,0,0,0,0,6


In [166]:
skin_df_age["age"] = pd.to_numeric(skin_df_metadata["characteristics..age"].tolist(), errors="coerce")

In [167]:
skin_df_age.head(3)

Gene,ENSMUSG00000109644,ENSMUSG00000007777,ENSMUSG00000043644,ENSMUSG00000042208,ENSMUSG00000020831,ENSMUSG00000107002,ENSMUSG00000058706,ENSMUSG00000099146,ENSMUSG00000028608,ENSMUSG00000097882,...,ENSMUSG00000096037,ENSMUSG00000095183,ENSMUSG00000094760,ENSMUSG00000095749,ENSMUSG00000094657,ENSMUSG00000095645,ENSMUSG00000096765,ENSMUSG00000096170,ENSMUSG00000095453,age
Skin_39-m-9,27,78,37,138,34,222,8,24,300,23,...,0,0,0,0,0,0,0,0,0,9
Skin_23-m-1,10,56,26,167,35,216,38,17,247,29,...,0,0,0,0,0,0,0,0,0,1
Skin_9-m-27,0,75,12,130,45,117,47,22,234,45,...,0,0,0,0,0,0,0,0,0,27


In [168]:
skin_log_transformed_data = np.log1p(skin_df_counts)

In [169]:
skin_data = skin_df_age.values

In [170]:
X = skin_log_transformed_data.values
y = skin_data[:,-1]

In [ ]:
## Define the parameter grid
param_grid = {
    'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1.0, 10.0, 100.0],
    'l1_ratio': np.arange(0, 1.1, 0.1)
}

# Define the model
model = ElasticNet(max_iter=10000, random_state=seed)

# Define Leave-One-Out cross-validation
loo = LeaveOneOut()

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=loo, scoring='neg_mean_absolute_error', n_jobs=-1)

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Fit the GridSearchCV
grid_search.fit(X_scaled, y)

# Get the best parameters
best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

# Train the model with the best parameters using Leave-One-Out cross-validation
y_true = []
y_pred = []

for train_index, test_index in loo.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    best_model = ElasticNet(alpha=best_params['alpha'], l1_ratio=best_params['l1_ratio'], max_iter=10000, random_state=seed)
    best_model.fit(X_train_scaled, y_train)
    
    y_test_pred = best_model.predict(X_test_scaled)
    
    y_true.append(y_test[0])
    y_pred.append(y_test_pred[0])

# Convert lists to numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Calculate the Pearson correlation coefficient
pearson_corr, _ = pearsonr(y_true, y_pred)

# Calculate the R-squared value
r_squared = r2_score(y_true, y_pred)

# Calculate the mean absolute error
mae = mean_absolute_error(y_true, y_pred)

print(f"Mean MAE: {mae:.3f}")
print(f"Pearson correlation: {pearson_corr:.3f}")
print(f"R-squared: {r_squared:.3f}")

In [171]:
loo = LeaveOneOut()

# Initialize lists to store actual and predicted values
y_true = []
y_pred = []

# Loop over each train-test split
for train_index, test_index in loo.split(X):
    # Split the data
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Scale the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Define and train the model
    model = ElasticNet(alpha=0.01, l1_ratio=1.0, max_iter=100000, random_state=seed)
    model.fit(X_train_scaled, y_train)
    #model.fit(X_train, y_train)
    
    # Make prediction on the test set
    y_test_pred = model.predict(X_test_scaled)
    #y_test_pred = model.predict(X_test)
    
    # Store the actual and predicted values
    y_true.append(y_test[0])
    y_pred.append(y_test_pred[0])

# Convert lists to numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Calculate the Pearson correlation coefficient
pearson_corr, _ = pearsonr(y_true, y_pred)

# Calculate the R-squared value
r_squared = r2_score(y_true, y_pred)

# Calculate the mean absolute error
mae = mean_absolute_error(y_true, y_pred)

print(f"Mean MAE: {mae:.3f}")
print(f"Pearson correlation: {pearson_corr:.3f}")
print(f"R-squared: {r_squared:.3f}")

Mean MAE: 2.965
Pearson correlation: 0.894
R-squared: 0.780


# Small Intestine

In [172]:
small_df_metadata = df_new_metadata[df_new_metadata["source.name"].str.startswith("Small")]

small_df_metadata.head(3)

,Unnamed: 0,Sample.name,title,source.name,organism,characteristics..age,characteristics..developmental.stage,characteristics..sex,molecule,description,processed.data.file,raw.file,BioSample,Instrument.Model
Sample.name.new,,,,,,,,,,,,,,
Small_Intestine_46-m-12,A11_384Bulk_Plate3_S11,A11_384Bulk_Plate3_S11,Tabula Muris Senis (bulk RNA seq),Small_Intestine_46,Mus musculus C57/BL6,12,months postnatal,m,total RNA,NaN,190214_A00111_0269_AHH3J3DSXX__190214_A00111_0...,SRR9127127,SAMN11854573,Illumina NovaSeq 6000
Small_Intestine_34-m-3,A14_384Bulk_Plate3_S14,A14_384Bulk_Plate3_S14,Tabula Muris Senis (bulk RNA seq),Small_Intestine_34,Mus musculus C57/BL6,3,months postnatal,m,total RNA,NaN,190214_A00111_0269_AHH3J3DSXX__190214_A00111_0...,SRR9127022,SAMN11854582,Illumina NovaSeq 6000
Small_Intestine_22-m-18,A17_384Bulk_Plate1_S17,A17_384Bulk_Plate1_S17,Tabula Muris Senis (bulk RNA seq),Small_Intestine_22,Mus musculus C57/BL6,18,months postnatal,m,total RNA,NaN,190214_A00111_0269_AHH3J3DSXX__190214_A00111_0...,SRR9127267,SAMN11854587,Illumina NovaSeq 6000


In [173]:
small_df_counts = df.loc[small_df_metadata.index.tolist(),]

small_df_counts.head(3)

Gene,ENSMUSG00000109644,ENSMUSG00000007777,ENSMUSG00000043644,ENSMUSG00000042208,ENSMUSG00000020831,ENSMUSG00000107002,ENSMUSG00000058706,ENSMUSG00000099146,ENSMUSG00000028608,ENSMUSG00000097882,...,ENSMUSG00000096015,ENSMUSG00000096037,ENSMUSG00000095183,ENSMUSG00000094760,ENSMUSG00000095749,ENSMUSG00000094657,ENSMUSG00000095645,ENSMUSG00000096765,ENSMUSG00000096170,ENSMUSG00000095453
Small_Intestine_46-m-12,477,215,12,93,14,537,40,2,193,0,...,0,0,0,0,0,0,0,0,0,0
Small_Intestine_34-m-3,639,179,18,59,17,552,24,3,148,3,...,0,0,0,0,0,0,0,0,0,0
Small_Intestine_22-m-18,760,187,12,101,34,470,90,2,238,4,...,0,0,0,0,0,0,0,0,0,0


In [174]:
small_df_age = small_df_counts.copy()

In [175]:
small_df_age["age"] = small_df_metadata["characteristics..age"].tolist()

In [176]:
small_df_age.head(5)

Gene,ENSMUSG00000109644,ENSMUSG00000007777,ENSMUSG00000043644,ENSMUSG00000042208,ENSMUSG00000020831,ENSMUSG00000107002,ENSMUSG00000058706,ENSMUSG00000099146,ENSMUSG00000028608,ENSMUSG00000097882,...,ENSMUSG00000096037,ENSMUSG00000095183,ENSMUSG00000094760,ENSMUSG00000095749,ENSMUSG00000094657,ENSMUSG00000095645,ENSMUSG00000096765,ENSMUSG00000096170,ENSMUSG00000095453,age
Small_Intestine_46-m-12,477,215,12,93,14,537,40,2,193,0,...,0,0,0,0,0,0,0,0,0,12
Small_Intestine_34-m-3,639,179,18,59,17,552,24,3,148,3,...,0,0,0,0,0,0,0,0,0,3
Small_Intestine_22-m-18,760,187,12,101,34,470,90,2,238,4,...,0,0,0,0,0,0,0,0,0,18
Small_Intestine_40-m-12,1065,101,22,100,18,326,54,7,128,0,...,0,0,0,0,0,0,0,0,0,12
Small_Intestine_51-m-9,769,86,9,77,14,283,21,3,115,3,...,0,0,0,0,0,0,0,0,0,9


In [177]:
small_df_age["age"] = pd.to_numeric(small_df_metadata["characteristics..age"].tolist(), errors="coerce")

In [178]:
small_df_age.head(3)

Gene,ENSMUSG00000109644,ENSMUSG00000007777,ENSMUSG00000043644,ENSMUSG00000042208,ENSMUSG00000020831,ENSMUSG00000107002,ENSMUSG00000058706,ENSMUSG00000099146,ENSMUSG00000028608,ENSMUSG00000097882,...,ENSMUSG00000096037,ENSMUSG00000095183,ENSMUSG00000094760,ENSMUSG00000095749,ENSMUSG00000094657,ENSMUSG00000095645,ENSMUSG00000096765,ENSMUSG00000096170,ENSMUSG00000095453,age
Small_Intestine_46-m-12,477,215,12,93,14,537,40,2,193,0,...,0,0,0,0,0,0,0,0,0,12
Small_Intestine_34-m-3,639,179,18,59,17,552,24,3,148,3,...,0,0,0,0,0,0,0,0,0,3
Small_Intestine_22-m-18,760,187,12,101,34,470,90,2,238,4,...,0,0,0,0,0,0,0,0,0,18


In [179]:
small_log_transformed_data = np.log1p(small_df_counts)

In [180]:
small_data = small_df_age.values

In [181]:
X = small_log_transformed_data.values
y = small_data[:,-1]

In [ ]:
## Define the parameter grid
param_grid = {
    'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1.0, 10.0, 100.0],
    'l1_ratio': np.arange(0, 1.1, 0.1)
}

# Define the model
model = ElasticNet(max_iter=10000, random_state=seed)

# Define Leave-One-Out cross-validation
loo = LeaveOneOut()

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=loo, scoring='neg_mean_absolute_error', n_jobs=-1)

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Fit the GridSearchCV
grid_search.fit(X_scaled, y)

# Get the best parameters
best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

# Train the model with the best parameters using Leave-One-Out cross-validation
y_true = []
y_pred = []

for train_index, test_index in loo.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    best_model = ElasticNet(alpha=best_params['alpha'], l1_ratio=best_params['l1_ratio'], max_iter=10000, random_state=seed)
    best_model.fit(X_train_scaled, y_train)
    
    y_test_pred = best_model.predict(X_test_scaled)
    
    y_true.append(y_test[0])
    y_pred.append(y_test_pred[0])

# Convert lists to numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Calculate the Pearson correlation coefficient
pearson_corr, _ = pearsonr(y_true, y_pred)

# Calculate the R-squared value
r_squared = r2_score(y_true, y_pred)

# Calculate the mean absolute error
mae = mean_absolute_error(y_true, y_pred)

print(f"Mean MAE: {mae:.3f}")
print(f"Pearson correlation: {pearson_corr:.3f}")
print(f"R-squared: {r_squared:.3f}")

In [182]:
loo = LeaveOneOut()

# Initialize lists to store actual and predicted values
y_true = []
y_pred = []

# Loop over each train-test split
for train_index, test_index in loo.split(X):
    # Split the data
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Scale the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Define and train the model
    model = ElasticNet(alpha=0.1, l1_ratio=1.0, max_iter=100000, random_state=seed)
    model.fit(X_train_scaled, y_train)
    #model.fit(X_train, y_train)
    
    # Make prediction on the test set
    y_test_pred = model.predict(X_test_scaled)
    #y_test_pred = model.predict(X_test)
    
    # Store the actual and predicted values
    y_true.append(y_test[0])
    y_pred.append(y_test_pred[0])

# Convert lists to numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Calculate the Pearson correlation coefficient
pearson_corr, _ = pearsonr(y_true, y_pred)

# Calculate the R-squared value
r_squared = r2_score(y_true, y_pred)

# Calculate the mean absolute error
mae = mean_absolute_error(y_true, y_pred)

print(f"Mean MAE: {mae:.3f}")
print(f"Pearson correlation: {pearson_corr:.3f}")
print(f"R-squared: {r_squared:.3f}")

Mean MAE: 1.899
Pearson correlation: 0.944
R-squared: 0.880


# Spleen

In [183]:
spleen_df_metadata = df_new_metadata[df_new_metadata["source.name"].str.startswith("Spleen")]

spleen_df_metadata.head(3)

,Unnamed: 0,Sample.name,title,source.name,organism,characteristics..age,characteristics..developmental.stage,characteristics..sex,molecule,description,processed.data.file,raw.file,BioSample,Instrument.Model
Sample.name.new,,,,,,,,,,,,,,
Spleen_7-f-12,A13_384Bulk_Plate1_S13,A13_384Bulk_Plate1_S13,Tabula Muris Senis (bulk RNA seq),Spleen_7,Mus musculus C57/BL6,12,months postnatal,f,total RNA,NaN,190214_A00111_0269_AHH3J3DSXX__190214_A00111_0...,SRR9127029,SAMN11854577,Illumina NovaSeq 6000
Spleen_50-m-12,A22_384Bulk_Plate1_S22,A22_384Bulk_Plate1_S22,Tabula Muris Senis (bulk RNA seq),Spleen_50,Mus musculus C57/BL6,12,months postnatal,m,total RNA,NaN,190214_A00111_0269_AHH3J3DSXX__190214_A00111_0...,SRR9127504,SAMN11854600,Illumina NovaSeq 6000
Spleen_30-m-3,A3_384Bulk_Plate1_S3,A3_384Bulk_Plate1_S3,Tabula Muris Senis (bulk RNA seq),Spleen_30,Mus musculus C57/BL6,3,months postnatal,m,total RNA,NaN,190214_A00111_0269_AHH3J3DSXX__190214_A00111_0...,SRR9127508,SAMN11854604,Illumina NovaSeq 6000


In [184]:
spleen_df_counts = df.loc[spleen_df_metadata.index.tolist(),]

spleen_df_counts.head(3)

Gene,ENSMUSG00000109644,ENSMUSG00000007777,ENSMUSG00000043644,ENSMUSG00000042208,ENSMUSG00000020831,ENSMUSG00000107002,ENSMUSG00000058706,ENSMUSG00000099146,ENSMUSG00000028608,ENSMUSG00000097882,...,ENSMUSG00000096015,ENSMUSG00000096037,ENSMUSG00000095183,ENSMUSG00000094760,ENSMUSG00000095749,ENSMUSG00000094657,ENSMUSG00000095645,ENSMUSG00000096765,ENSMUSG00000096170,ENSMUSG00000095453
Spleen_7-f-12,9,30,21,61,49,193,56,4,328,7,...,0,0,0,0,0,0,0,0,0,0
Spleen_50-m-12,4,63,28,151,61,225,101,0,357,8,...,0,0,0,0,0,0,0,0,0,0
Spleen_30-m-3,7,72,44,196,106,250,92,0,652,13,...,0,0,0,0,0,0,0,0,0,0


In [185]:
spleen_df_age = spleen_df_counts.copy()

In [186]:
spleen_df_age["age"] = spleen_df_metadata["characteristics..age"].tolist()

In [187]:
spleen_df_age.head(5)

Gene,ENSMUSG00000109644,ENSMUSG00000007777,ENSMUSG00000043644,ENSMUSG00000042208,ENSMUSG00000020831,ENSMUSG00000107002,ENSMUSG00000058706,ENSMUSG00000099146,ENSMUSG00000028608,ENSMUSG00000097882,...,ENSMUSG00000096037,ENSMUSG00000095183,ENSMUSG00000094760,ENSMUSG00000095749,ENSMUSG00000094657,ENSMUSG00000095645,ENSMUSG00000096765,ENSMUSG00000096170,ENSMUSG00000095453,age
Spleen_7-f-12,9,30,21,61,49,193,56,4,328,7,...,0,0,0,0,0,0,0,0,0,12
Spleen_50-m-12,4,63,28,151,61,225,101,0,357,8,...,0,0,0,0,0,0,0,0,0,12
Spleen_30-m-3,7,72,44,196,106,250,92,0,652,13,...,0,0,0,0,0,0,0,0,0,3
Spleen_5-m-12,2,42,8,86,65,139,50,4,256,2,...,0,0,0,0,0,0,0,0,0,12
Spleen_21-f-21,14,91,13,138,38,171,50,3,445,10,...,0,0,0,0,0,0,0,0,0,21


In [188]:
spleen_df_age["age"] = pd.to_numeric(spleen_df_metadata["characteristics..age"].tolist(), errors="coerce")

In [189]:
spleen_df_age.head(3)

Gene,ENSMUSG00000109644,ENSMUSG00000007777,ENSMUSG00000043644,ENSMUSG00000042208,ENSMUSG00000020831,ENSMUSG00000107002,ENSMUSG00000058706,ENSMUSG00000099146,ENSMUSG00000028608,ENSMUSG00000097882,...,ENSMUSG00000096037,ENSMUSG00000095183,ENSMUSG00000094760,ENSMUSG00000095749,ENSMUSG00000094657,ENSMUSG00000095645,ENSMUSG00000096765,ENSMUSG00000096170,ENSMUSG00000095453,age
Spleen_7-f-12,9,30,21,61,49,193,56,4,328,7,...,0,0,0,0,0,0,0,0,0,12
Spleen_50-m-12,4,63,28,151,61,225,101,0,357,8,...,0,0,0,0,0,0,0,0,0,12
Spleen_30-m-3,7,72,44,196,106,250,92,0,652,13,...,0,0,0,0,0,0,0,0,0,3


In [190]:
spleen_log_transformed_data = np.log1p(spleen_df_counts)

In [191]:
spleen_data = spleen_df_age.values

In [192]:
X = spleen_log_transformed_data.values
y = spleen_data[:,-1]

In [ ]:
## Define the parameter grid
param_grid = {
    'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1.0, 10.0, 100.0],
    'l1_ratio': np.arange(0, 1.1, 0.1)
}

# Define the model
model = ElasticNet(max_iter=10000, random_state=seed)

# Define Leave-One-Out cross-validation
loo = LeaveOneOut()

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=loo, scoring='neg_mean_absolute_error', n_jobs=-1)

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Fit the GridSearchCV
grid_search.fit(X_scaled, y)

# Get the best parameters
best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

# Train the model with the best parameters using Leave-One-Out cross-validation
y_true = []
y_pred = []

for train_index, test_index in loo.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    best_model = ElasticNet(alpha=best_params['alpha'], l1_ratio=best_params['l1_ratio'], max_iter=10000, random_state=seed)
    best_model.fit(X_train_scaled, y_train)
    
    y_test_pred = best_model.predict(X_test_scaled)
    
    y_true.append(y_test[0])
    y_pred.append(y_test_pred[0])

# Convert lists to numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Calculate the Pearson correlation coefficient
pearson_corr, _ = pearsonr(y_true, y_pred)

# Calculate the R-squared value
r_squared = r2_score(y_true, y_pred)

# Calculate the mean absolute error
mae = mean_absolute_error(y_true, y_pred)

print(f"Mean MAE: {mae:.3f}")
print(f"Pearson correlation: {pearson_corr:.3f}")
print(f"R-squared: {r_squared:.3f}")

In [194]:
loo = LeaveOneOut()

# Initialize lists to store actual and predicted values
y_true = []
y_pred = []

# Loop over each train-test split
for train_index, test_index in loo.split(X):
    # Split the data
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Scale the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Define and train the model
    model = ElasticNet(alpha=0.001, l1_ratio=1.0, max_iter=100000, random_state=seed)
    model.fit(X_train_scaled, y_train)
    #model.fit(X_train, y_train)
    
    # Make prediction on the test set
    y_test_pred = model.predict(X_test_scaled)
    #y_test_pred = model.predict(X_test)
    
    # Store the actual and predicted values
    y_true.append(y_test[0])
    y_pred.append(y_test_pred[0])

# Convert lists to numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Calculate the Pearson correlation coefficient
pearson_corr, _ = pearsonr(y_true, y_pred)

# Calculate the R-squared value
r_squared = r2_score(y_true, y_pred)

# Calculate the mean absolute error
mae = mean_absolute_error(y_true, y_pred)

print(f"Mean MAE: {mae:.3f}")
print(f"Pearson correlation: {pearson_corr:.3f}")
print(f"R-squared: {r_squared:.3f}")

Mean MAE: 2.558
Pearson correlation: 0.917
R-squared: 0.823


# WBC

In [195]:
wbc_df_metadata = df_new_metadata[df_new_metadata["source.name"].str.startswith("WBC")]

wbc_df_metadata.head(3)

,Unnamed: 0,Sample.name,title,source.name,organism,characteristics..age,characteristics..developmental.stage,characteristics..sex,molecule,description,processed.data.file,raw.file,BioSample,Instrument.Model
Sample.name.new,,,,,,,,,,,,,,
WBC_34-f-9,A11_384Bulk_Plate2_S11,A11_384Bulk_Plate2_S11,Tabula Muris Senis (bulk RNA seq),WBC_34,Mus musculus C57/BL6,9,months postnatal,f,total RNA,NaN,190214_A00111_0269_AHH3J3DSXX__190214_A00111_0...,SRR9127107,SAMN11854572,Illumina NovaSeq 6000
WBC_30-m-12,A19_384Bulk_Plate1_S19,A19_384Bulk_Plate1_S19,Tabula Muris Senis (bulk RNA seq),WBC_30,Mus musculus C57/BL6,12,months postnatal,m,total RNA,NaN,190214_A00111_0269_AHH3J3DSXX__190214_A00111_0...,SRR9127071,SAMN11854591,Illumina NovaSeq 6000
WBC_22-m-12,A23_384Bulk_Plate1_S23,A23_384Bulk_Plate1_S23,Tabula Muris Senis (bulk RNA seq),WBC_22,Mus musculus C57/BL6,12,months postnatal,m,total RNA,NaN,190214_A00111_0269_AHH3J3DSXX__190214_A00111_0...,SRR9127506,SAMN11854602,Illumina NovaSeq 6000


In [196]:
wbc_df_counts = df.loc[wbc_df_metadata.index.tolist(),]

wbc_df_counts.head(3)

Gene,ENSMUSG00000109644,ENSMUSG00000007777,ENSMUSG00000043644,ENSMUSG00000042208,ENSMUSG00000020831,ENSMUSG00000107002,ENSMUSG00000058706,ENSMUSG00000099146,ENSMUSG00000028608,ENSMUSG00000097882,...,ENSMUSG00000096015,ENSMUSG00000096037,ENSMUSG00000095183,ENSMUSG00000094760,ENSMUSG00000095749,ENSMUSG00000094657,ENSMUSG00000095645,ENSMUSG00000096765,ENSMUSG00000096170,ENSMUSG00000095453
WBC_34-f-9,3,2,9,41,8,31,21,3,78,0,...,0,0,0,0,0,0,0,0,0,0
WBC_30-m-12,0,8,16,12,8,20,8,0,53,2,...,0,0,0,0,0,0,0,0,0,0
WBC_22-m-12,0,0,10,1,0,3,0,0,13,3,...,0,0,0,0,0,0,0,0,0,0


In [197]:
wbc_df_age = wbc_df_counts.copy()

In [198]:
wbc_df_age["age"] = wbc_df_metadata["characteristics..age"].tolist()

In [199]:
wbc_df_age.head(5)

Gene,ENSMUSG00000109644,ENSMUSG00000007777,ENSMUSG00000043644,ENSMUSG00000042208,ENSMUSG00000020831,ENSMUSG00000107002,ENSMUSG00000058706,ENSMUSG00000099146,ENSMUSG00000028608,ENSMUSG00000097882,...,ENSMUSG00000096037,ENSMUSG00000095183,ENSMUSG00000094760,ENSMUSG00000095749,ENSMUSG00000094657,ENSMUSG00000095645,ENSMUSG00000096765,ENSMUSG00000096170,ENSMUSG00000095453,age
WBC_34-f-9,3,2,9,41,8,31,21,3,78,0,...,0,0,0,0,0,0,0,0,0,9
WBC_30-m-12,0,8,16,12,8,20,8,0,53,2,...,0,0,0,0,0,0,0,0,0,12
WBC_22-m-12,0,0,10,1,0,3,0,0,13,3,...,0,0,0,0,0,0,0,0,0,12
WBC_52-m-21,0,12,24,10,39,41,18,0,49,0,...,0,0,0,0,0,0,0,0,0,21
WBC_6-m-18,1,2,7,6,5,12,0,0,11,0,...,0,0,0,0,0,0,0,0,0,18


In [200]:
wbc_df_age["age"] = pd.to_numeric(wbc_df_metadata["characteristics..age"].tolist(), errors="coerce")

In [201]:
wbc_df_age.head(3)

Gene,ENSMUSG00000109644,ENSMUSG00000007777,ENSMUSG00000043644,ENSMUSG00000042208,ENSMUSG00000020831,ENSMUSG00000107002,ENSMUSG00000058706,ENSMUSG00000099146,ENSMUSG00000028608,ENSMUSG00000097882,...,ENSMUSG00000096037,ENSMUSG00000095183,ENSMUSG00000094760,ENSMUSG00000095749,ENSMUSG00000094657,ENSMUSG00000095645,ENSMUSG00000096765,ENSMUSG00000096170,ENSMUSG00000095453,age
WBC_34-f-9,3,2,9,41,8,31,21,3,78,0,...,0,0,0,0,0,0,0,0,0,9
WBC_30-m-12,0,8,16,12,8,20,8,0,53,2,...,0,0,0,0,0,0,0,0,0,12
WBC_22-m-12,0,0,10,1,0,3,0,0,13,3,...,0,0,0,0,0,0,0,0,0,12


In [202]:
wbc_log_transformed_data = np.log1p(wbc_df_counts)

In [203]:
wbc_data = wbc_df_age.values

In [204]:
X = wbc_log_transformed_data.values
y = wbc_data[:,-1]

In [ ]:
## Define the parameter grid
param_grid = {
    'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1.0, 10.0, 100.0],
    'l1_ratio': np.arange(0, 1.1, 0.1)
}

# Define the model
model = ElasticNet(max_iter=10000, random_state=seed)

# Define Leave-One-Out cross-validation
loo = LeaveOneOut()

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=loo, scoring='neg_mean_absolute_error', n_jobs=-1)

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Fit the GridSearchCV
grid_search.fit(X_scaled, y)

# Get the best parameters
best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

# Train the model with the best parameters using Leave-One-Out cross-validation
y_true = []
y_pred = []

for train_index, test_index in loo.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    best_model = ElasticNet(alpha=best_params['alpha'], l1_ratio=best_params['l1_ratio'], max_iter=10000, random_state=seed)
    best_model.fit(X_train_scaled, y_train)
    
    y_test_pred = best_model.predict(X_test_scaled)
    
    y_true.append(y_test[0])
    y_pred.append(y_test_pred[0])

# Convert lists to numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Calculate the Pearson correlation coefficient
pearson_corr, _ = pearsonr(y_true, y_pred)

# Calculate the R-squared value
r_squared = r2_score(y_true, y_pred)

# Calculate the mean absolute error
mae = mean_absolute_error(y_true, y_pred)

print(f"Mean MAE: {mae:.3f}")
print(f"Pearson correlation: {pearson_corr:.3f}")
print(f"R-squared: {r_squared:.3f}")

In [205]:
loo = LeaveOneOut()

# Initialize lists to store actual and predicted values
y_true = []
y_pred = []

# Loop over each train-test split
for train_index, test_index in loo.split(X):
    # Split the data
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Scale the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Define and train the model
    model = ElasticNet(alpha=0.01, l1_ratio=0.6000000000000001, max_iter=100000, random_state=seed)
    model.fit(X_train_scaled, y_train)
    #model.fit(X_train, y_train)
    
    # Make prediction on the test set
    y_test_pred = model.predict(X_test_scaled)
    #y_test_pred = model.predict(X_test)
    
    # Store the actual and predicted values
    y_true.append(y_test[0])
    y_pred.append(y_test_pred[0])

# Convert lists to numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Calculate the Pearson correlation coefficient
pearson_corr, _ = pearsonr(y_true, y_pred)

# Calculate the R-squared value
r_squared = r2_score(y_true, y_pred)

# Calculate the mean absolute error
mae = mean_absolute_error(y_true, y_pred)

print(f"Mean MAE: {mae:.3f}")
print(f"Pearson correlation: {pearson_corr:.3f}")
print(f"R-squared: {r_squared:.3f}")

Mean MAE: 2.496
Pearson correlation: 0.923
R-squared: 0.829
